In [1]:
import pandas as pd
# https://gtuckerkellogg.github.io/pairwise/demo/ for debugging

In [7]:
def getScoring():
    with open('blosum62.txt') as matrix_file:
        matrix = matrix_file.read()
        lines = matrix.strip().split('\n')

    blosum={}
    cols = lines[0].split()

    for row in lines[1:]:       
        idx = row[0]
        vals = row[1:].split()
        vals = [int(_) for _ in vals]
        #print(idx,vals)
        blosum[idx]=vals

    blosum = pd.DataFrame.from_dict(blosum,orient='index',columns=cols)
    return blosum

def scorematch(a,b,scoringMatrix):
    return scoringMatrix.loc[a][b]

In [3]:
blosum = getScoring()
blosum.head()

,A,R,N,D,C,Q,E,G,H,I,...,P,S,T,W,Y,V,B,Z,X,*
A,4,-1,-2,-2,0,-1,-1,0,-2,-1,...,-1,1,0,-3,-2,0,-2,-1,0,-4
R,-1,5,0,-2,-3,1,0,-2,0,-3,...,-2,-1,-1,-3,-2,-3,-1,0,-1,-4
N,-2,0,6,1,-3,0,0,0,1,-3,...,-2,1,0,-4,-2,-3,3,0,-1,-4
D,-2,-2,1,6,-3,0,2,-1,-1,-3,...,-1,0,-1,-4,-3,-3,4,1,-1,-4
C,0,-3,-3,-3,9,-3,-4,-3,-3,-1,...,-3,-1,-1,-2,-2,-1,-3,-3,-2,-4


In [8]:
scorematch("A",'A',blosum)

4

In [115]:
scorematch("C",'C')

9

In [132]:
x='PLEASANTLY'
y='MEANLY'
#match = 1
indel = -5
#mismatch = -1


m = len(x)
n = len(y)
Fmatrix = np.zeros((m+1,n+1))

## initialize
for i in range(1,m+1):
    Fmatrix[i,0] += Fmatrix[i-1,0] + indel
for j in range(1,n+1):
    Fmatrix[0,j] += Fmatrix[0,j-1] + indel

##
for i in range(1,m+1):
    for j in range(1,n+1):a
        if x[i-1] == y[j-1]:
            Fmatrix[i,j] = Fmatrix[i-1,j-1] + scorematch(x[i-1],y[j-1])
        else:
            mm = Fmatrix[i-1,j-1] + scorematch(x[i-1],y[j-1])
            down = Fmatrix[i-1,j] + indel
            right = Fmatrix[i,j-1] + indel
            Fmatrix[i,j] = max([mm,down,right])
        #print(x[i-1],y[j-1],[mm,down,right])
            
            
pd.DataFrame(Fmatrix,columns=['-']+[_ for _ in y],index=['-']+[_ for _ in x])

,-,M,E,A,N,L,Y
-,0.0,-5.0,-10.0,-15.0,-20.0,-25.0,-30.0
P,-5.0,-2.0,-6.0,-11.0,-16.0,-21.0,-26.0
L,-10.0,-3.0,-5.0,-7.0,-12.0,-12.0,-17.0
E,-15.0,-8.0,2.0,-3.0,-7.0,-12.0,-14.0
A,-20.0,-13.0,-3.0,6.0,1.0,-4.0,-9.0
S,-25.0,-18.0,-8.0,1.0,7.0,2.0,-3.0
A,-30.0,-23.0,-13.0,-4.0,2.0,6.0,1.0
N,-35.0,-28.0,-18.0,-9.0,2.0,1.0,4.0
T,-40.0,-33.0,-23.0,-14.0,-3.0,1.0,-1.0
L,-45.0,-38.0,-28.0,-19.0,-8.0,1.0,0.0


In [153]:
i = Fmatrix.shape[0] - 1
j = Fmatrix.shape[1] - 1

LCS = ""
a = ''
b = ''
paths = []
while (j>0 or i>0):
    if x[i-1] == y[j-1]:
        print(x[i-1],y[j-1],Fmatrix[i,j])
        LCS += x[i-1]
        j -=1
        i -=1
        a +=x[i]
        b +=y[j]

        paths.append([(np.inf),(x[i],y[j]) ])
    else:
        diag = Fmatrix[i-1,j-1] + scorematch(x[i-1],y[j-1])
        left = Fmatrix[i-1,j] + indel
        up = Fmatrix[i,j-1] + indel
        maxCell = np.argmax([diag,left,up])
        print(maxCell,'diff',[diag,left,up])
        if maxCell == 0:
            j -=1
            i -=1
            a +=x[i]
            b +=y[j]
        else:
            if maxCell == 1:
                i -= 1
                a += x[i]
                b += '-'
            else:
                j -= 1
                a += '-'
                b += y[j]
        paths.append( [(diag,left,up),
                       (x[i],x[i],"-"),
                       (y[j],"-",y[j]) ])

        
print('Score',Fmatrix[m][n])        
print(LCS[::-1]) #AACTGG
print('--')
print(b[::-1])
print(a[::-1])

Y Y 8.0
L L 1.0
1 diff [-9.0, -3.0, -19.0]
N N 2.0
A A -4.0
1 diff [-13.0, -8.0, -23.0]
1 diff [-9.0, -3.0, -18.0]
E E 2.0
0 diff [-3.0, -7.0, -15.0]
1 diff [-33.0, -5.0, -31.0]
Score 8.0
EANLY
--
-ME--AN-LY
PLEASANTLY


In [281]:
def globalAlignment(x,y,indel=-5):
    m = len(x)
    n = len(y)
    Fmatrix = np.zeros((m+1,n+1))

    ## initialize
    for i in range(1,m+1):
        Fmatrix[i,0] += Fmatrix[i-1,0] + indel
    for j in range(1,n+1):
        Fmatrix[0,j] += Fmatrix[0,j-1] + indel

    ##
    for i in range(1,m+1):
        for j in range(1,n+1):
            if x[i-1] == y[j-1]:
                Fmatrix[i,j] = Fmatrix[i-1,j-1] + scorematch(x[i-1],y[j-1])
            else:
                mm = Fmatrix[i-1,j-1] + scorematch(x[i-1],y[j-1])
                down = Fmatrix[i-1,j] + indel
                right = Fmatrix[i,j-1] + indel
                Fmatrix[i,j] = max([mm,down,right])
            #print(x[i-1],y[j-1],[mm,down,right])

    #pd.DataFrame(Fmatrix,columns=['-']+[_ for _ in y],index=['-']+[_ for _ in x])
    ##
    i = Fmatrix.shape[0] - 1
    j = Fmatrix.shape[1] - 1

    LCS = ""
    a = ''
    b = ''
    paths = []
    while (j>0 or i>0):
        if x[i-1] == y[j-1]:
            #print(x[i-1],y[j-1],Fmatrix[i,j])
            LCS += x[i-1]
            j -=1
            i -=1
            a +=x[i]
            b +=y[j]

            paths.append([(np.inf),(x[i],y[j]) ])
        else:
            diag = Fmatrix[i-1,j-1] + scorematch(x[i-1],y[j-1])
            left = Fmatrix[i-1,j] + indel
            up = Fmatrix[i,j-1] + indel
            maxCell = np.argmax([diag,left,up])
            #print(maxCell,'diff',[diag,left,up])
            if maxCell == 0:
                j -=1
                i -=1
                a +=x[i]
                b +=y[j]
            else:
                if maxCell == 1:
                    i -= 1
                    a += x[i]
                    b += '-'
                else:
                    j -= 1
                    a += '-'
                    b += y[j]
            paths.append( [(diag,left,up),
                           (x[i],x[i],"-"),
                           (y[j],"-",y[j]) ])


    #print('Score',Fmatrix[m][n])        
    #print(LCS[::-1]) #AACTGG
    #print('--')
    #print(b[::-1])
    #print(a[::-1])
    score = Fmatrix[m][n]
    LCS = LCS[::-1]
    a = a[::-1]
    b = b[::-1]
    return score, LCS, a,b,Fmatrix

In [282]:
x='PLEASANTLY'
y='MEANLY'
alScore, LCS, a,b,_ = globalAlignment(x,y)

print(alScore)
print(a)
print(b)
# 8
# PLEASANTLY
# -MEA--N-LY

8.0
PLEASANTLY
-ME--AN-LY


In [283]:
###http://bioinformaticsalgorithms.com/data/extradatasets/alignment/global_alignment.txt
x='ILYPRQSMICMSFCFWDMWKKDVPVVLMMFLERRQMQSVFSWLVTVKTDCGKGIYNHRKYLGLPTMTAGDWHWIKKQNDPHEWFQGRLETAWLHSTFLYWKYFECDAVKVCMDTFGLFGHCDWDQQIHTCTHENEPAIAFLDLYCRHSPMCDKLYPVWDMACQTCHFHHSWFCRNQEMWMKGDVDDWQWGYHYHTINSAQCNQWFKEICKDMGWDSVFPPRHNCQRHKKCMPALYAGIWMATDHACTFMVRLIYTENIAEWHQVYCYRSMNMFTCGNVCLRCKSWIFVKNYMMAPVVNDPMIEAFYKRCCILGKAWYDMWGICPVERKSHWEIYAKDLLSFESCCSQKKQNCYTDNWGLEYRLFFQSIQMNTDPHYCQTHVCWISAMFPIYSPFYTSGPKEFYMWLQARIDQNMHGHANHYVTSGNWDSVYTPEKRAGVFPVVVPVWYPPQMCNDYIKLTYECERFHVEGTFGCNRWDLGCRRYIIFQCPYCDTMKICYVDQWRSIKEGQFRMSGYPNHGYWFVHDDHTNEWCNQPVLAKFVRSKIVAICKKSQTVFHYAYTPGYNATWPQTNVCERMYGPHDNLLNNQQNVTFWWKMVPNCGMQILISCHNKMKWPTSHYVFMRLKCMHVLMQMEYLDHFTGPGEGDFCRNMQPYMHQDLHWEGSMRAILEYQAEHHRRAFRAELCAQYDQEIILWSGGWGVQDCGFHANYDGSLQVVSGEPCSMWCTTVMQYYADCWEKCMFA'
y='ILIPRQQMGCFPFPWHFDFCFWSAHHSLVVPLNPQMQTVFQNRGLDRVTVKTDCHDHRWKWIYNLGLPTMTAGDWHFIKKHVVRANNPHQWFQGRLTTAWLHSTFLYKKTEYCLVRHSNCCHCDWDQIIHTCAFIAFLDLYQRHWPMCDKLYCHFHHSWFCRNQEMSMDWNQWFPWDSVPRANCLEEGALIALYAGIWANSMKRDMKTDHACTVRLIYVCELHAWLKYCYTSINMLCGNVCLRCKSWIFVKLFYMYAPVVNTIEANSPHYYKRCCILGQGICPVERKSHCEIYAKDLLSFESCCSQKQNCYTDNWGLEYRLFFQHIQMECTDPHANRGWTSCQTAKYWHFNLDDRPPKEFYMWLQATPTDLCMYQHCLMFKIVKQNFRKQHGHANPAASTSGNWDSVYTPEKMAYKDWYVSHPPVDMRRNGSKMVPVWYPPGIWHWKQSYKLTYECFFTVPGRFHVEGTFGCNRWDHQPGTRRDRQANHQFQCPYSDTMAIWEHAYTYVDQWRSIKEGQMPMSGYPNHGQWNVHDDHTNEQERSPICNQPVLAKFVRSKNVSNHEICKKSQTVFHWACEAQTNVCERMLNNQHVAVKRNVTFWWQMVPNCLWSCHNKMTWPTRPEQHRLFFVKMRLKCMHEYLDVAPSDFCRNMQAYMHSMRAILEYQADFDLKRRLRAIAPMDLCAQYDQEIILWSGGYIYDQSLQVVSCEGCSYYADCYVKCINVKEKCMFA'
alScore, LCS, a,b,_ = globalAlignment(x,y)

print(alScore)
print(a)
print(b)
# 1555
# ILYPRQSMICMSFCF-WDM--WKKDVPVVLMMFLERRQMQSVF-S-WL--VTVKTDCGKGIYNHR-K--Y-LGLPTMTAGDWHWIKK---Q-NDPHEWFQGRLETAWLHSTFLYWKYFE-CDAVKVCMDTFGLFGHCDWDQQIHTCTHENEPAIAFLDLYCRHSPMCDKLYPVWDMACQTCHFHHSWFCRNQEMWMKGDVDDWQWGYHYHTINSAQCNQWFKEICKDMGWDSVFPPRHNCQRHKKCMPALYAGIW---MA----TDHACTFMVRLIYTENIAEWHQVYCYRSMNMFTCGNVCLRCKSWIFVKN-YMMAPVVNDPMIEA--FYKRCCILGKAWYDMWGICPVERKSHWEIYAKDLLSFESCCSQKKQNCYTDNWGLEYRLFFQSIQMN-TDPH----Y--CQTHVCW-ISAMF-PIYSPFYT--SG-PKEFYMW---LQARI-DQNM---HGHANHYV-TSGNWDSVYTPEKRA--G--V-FP-V-V-------VPVWYPPQMCN--D-YIKLTYEC--E---RFHVEGTFGCNRWD-L-GCRR--YII--FQCPYCDTMKI---CY--VDQWRSIKEGQFRMSGYPNHGYWFVHDDHTNEW-----CNQPVLAKFVRSKIVA---ICKKSQTVFHYAYTPGYNATWPQTNVCERMYGPHDNLLNNQQNVTFWWKMVPNCGMQILISCHNKMKWPT--S-HYVF---MRLKCMHVLMQMEYLDHFTGPGEGDFCRNMQPYMHQDLHWEGSMRAILEYQAEHH-RRAFRA----ELCAQYDQEIILWSGGWGVQDCGFHANYDGSLQVVSGEPCSMWCTTVMQYYADCWEKCMFA
# ILIPRQQMGCFPFPWHFDFCFWSAHHSLVVP--LNP-QMQTVFQNRGLDRVTVKTDC----HDHRWKWIYNLGLPTMTAGDWHFIKKHVVRANNPHQWFQGRLTTAWLHSTFLY-KKTEYC-LVR---HS-NCC-HCDWDQIIHTCAF-----IAFLDLYQRHWPMCDKLY------C---HFHHSWFCRNQEMSM--D---W---------N--Q---WFP-------WDSV-P-RANCLE-EGALIALYAGIWANSMKRDMKTDHACT--VRLIYVCELHAWLK-YCYTSINML-CGNVCLRCKSWIFVKLFYMYAPVVNTIEANSPHYYKRCCILGQ------GICPVERKSHCEIYAKDLLSFESCCSQK-QNCYTDNWGLEYRLFFQHIQMECTDPHANRGWTSCQTAKYWHFNLDDRPP-KEFYMWLQATPTDLCMYQHCLMFKIVKQNFRKQHGHANPAASTSGNWDSVYTPEKMAYKDWYVSHPPVDMRRNGSKMVPVWYPPGIWHWKQSY-KLTYECFFTVPGRFHVEGTFGCNRWDHQPGTRRDRQANHQFQCPYSDTMAIWEHAYTYVDQWRSIKEGQMPMSGYPNHGQWNVHDDHTNEQERSPICNQPVLAKFVRSKNVSNHEICKKSQTVFHWA-C---EA---QTNVCERMLN-NQHVAV-KRNVTFWWQMVPNC----LWSCHNKMTWPTRPEQHRLFFVKMRLKCMH-----EYLD--VAPS--DFCRNMQAYMH-------SMRAILEYQADFDLKRRLRAIAPMDLCAQYDQEIILWSGGY-I--------YDQSLQVVSCEGCSYYADCYVKCI-NVKEKCMFA

1555.0
ILYPRQSMICMSFCF-WD--MWKKDVPVVLMMFLERRQMQSVF-S-WL--VTVKTDCGKGIYNHR-K--Y-LGLPTMTAGDWHWIKK---Q-NDPHEWFQGRLETAWLHSTFLYWKYFE-CDAVKVCMDTFGLFGHCDWDQQIHTCTHENEPAIAFLDLYCRHSPMCDKLYPVWDMACQTCHFHHSWFCRNQEMWMKGDVDDWQWGYHYHTINSAQCNQWFKEICKDMGWDSVFPPRHNCQRHKKCMPALYAGIW-------MATDHACTFMVRLIYTENIAEWHQVYCYRSMNMFTCGNVCLRCKSWIFVK-NYMMAPVVN--DPMIEAFYKRCCILGKAWYDMWGICPVERKSHWEIYAKDLLSFESCCSQKKQNCYTDNWGLEYRLFFQSIQMN-TDPH----Y--CQTHVCW-ISA-MFPIYSPFY--TSG-PKE--FYM-WLQARI-DQN---MHGHANHYV-TSGNWDSVYTPEKRA--G--V-F-PV--------VVPVWYPPQMCN--DYIKLTYEC----E-RFHVEGTFGCNRWD-L-GC-R-RYI--IFQCPYCDTMKI---C--YVDQWRSIKEGQFRMSGYPNHGYWFVHDDHTNEW-----CNQPVLAKFVRSKIVA---ICKKSQTVFHYAYTPGYNATWPQTNVCERMYGPHDNLLNNQQNVTFWWKMVPNCGMQILISCHNKMKWPT---SHYV-F--MRLKCMHVLMQMEYLDHFTGPGEGDFCRNMQPYMHQDLHWEGSMRAILEYQAEHH-RRAFR--A--ELCAQYDQEIILWSGGWGVQDCGFHANYDGSLQVVSGEPCSMWCTTVMQYYADCWEKCMFA
ILIPRQQMGCFPFPWHFDFCFWSAHHSLV-VP-L-NPQMQTVFQNRGLDRVTVKTDC----HDHRWKWIYNLGLPTMTAGDWHFIKKHVVRANNPHQWFQGRLTTAWLHSTFLY-KKTEYC-LVR---HS-NCC-HCDWDQIIHTCAF--

In [284]:
%time
x='FGVLMCLSDLFQYQRNSAEAYTDTAKWAYQIYQIVRHGFFPPCTNPSHAWYKRYYHVEYTEGNIQDITRWTHEWRRSYQPMMRAQPMHWAECTHRCCSSYPICCPKWNVFNIKTLYKMLYCHDGYNCNHRLVVRSLEHGCFRFLLKNNDVERTKHCIYGMIYNPPPNVMSYIPRGMCNSYHSMIQQSLVSTELHQGDQFVFNPNIYYYGMVFKLLYQFMSGPILVPMPTICTDHYQMSCRCADMNSICWRAHDPQAEQGPNKSHCDSMKFVNNYAHEPDRDILSTQCKDASYPERNNQGTGNVDQGCQKKNEPYQLKTWKVFHTMRGYGITQYNMPHRMHFEVVNWYWEVHSIFDAVRFEYYEIMRRSWWFTFSFTQSWNNCTDFLKLYQTWCRLGGLHHHMHENNYDCEWMVMCDKKVDVIIDLREPLRICWLADAKYFKEDNKRCYSHKAHHIAKCNFQVERGEGHPALNAWYGFMDSKVMLFRQAGEPFRVMLVHPDMHYARIDYDCACWGMGWNRYGCITQSKMCESEKHFEAHQMNRPTIKCRKNMLRYEMVRRTWTLLGREINVFLHVYCYSHAWHRIGKVFWAQFHVDTITRHTLLAWTMLHTIIHRIWHGHQFDNKQLNRIIKKSCRTMLTIFGMRNEFHKWAGFKWSQYRWNVIMHAHYSYKDHGAEISHVRWVTTFGSAGEVFTNPGLNYWLPDDLGCNVDPYHHGSSVAMAHRACQNMTTRLGNVEKRMGDMDMKSNLDQRDRIGNMEVNAVRWMAGKSHTSRACKREAGEWTSFEWEVDQYVHKGYNPMDLWKGRKPSLGWNTGHFAMVENEHDVWIACVWDSMHHNADKRGFKLFFTNRLMHVWDECKQYRYFSNKQQFIWMSLSINLTSHSLDISKSYWVFKHGWMTRVLVCNQNYQQECMPDNDPMSNHPIDYRFAPVHCIMCVHNQYTHPTVGHWYNMQWSWKVNCDVGDCSYWSVKAMETGFGATVQCDLADQDRRATPGSKTMCECPKLWNTNYYRYMRSQDEWDMWCDLAYLRFYDSSHYFRKIFQDRQDEEWKEFSAWCMWQRVIVFKLCIDFCIMQREMGGVTMYYTVDAPVGIRTYVDWEFIATRAAWNLENDILYMMPQGHNWDMDHCWVKGNRIQMAACVCQHIAELQTTIQGSSNHPFLQGRMLEFEQSSAQQRLTAFNMDGYVQYILVKYRWITCACTQPTQHMWWSRFKNPTKNMFNCNKEKEFPKGYYSEGPRFDDSRMNWSFDYHANQVIMQRWGHDSGKSRLKVRHCQSFHLTSFDDIINDQAVKQMYWWAYVKCTQEDEHKQVDWCPLKYSFVCRVAPKCYYIPVDCYQDAWYPNPQRNSSFSHSVMWRTDYNENRSKSFWQILQNPKLSFVRLEKYFHFQPFGEPPSPIGCATCIYPNHYAERANNQCVYESKSVSFFNQMCYCKRMLFPLGWWAWMGWDWWHLYIKKPVCEIDQILLEPWISNLEPSGRANLMAVIGCWAHLHRSEWRDQRLYHFKSCEQRKMPQPDCMCRIQCIWPIVVQGNHFIVQKGMWIMFKGMNRCESQLNRSYSHDVIEGNWQWCPFFALSKYEKNMAHWKNYKTLSRLCVYNVRCSEGRMHVPPMCKPPIWPPFRRYPVSYAIVATFRRQWYGTNVLSWVHGFRQQCDLWMMDAMSYWAAFWCGWSSVDCFGKTDNHASWTRGGERTEAVCFAMFAFVDWRNKIDRATEDRYTIVFVWRYWMVGLKNENGHKCHQNTKKIKRLFWECMLWTMLLTWTDRCINAVSRYQPIELNVNLCPITLDCDPRYQTYWQLNETELYFLAQQHFDLLCKFYYCYLNPQCVQHKVQENTTQCILFDYEWPQAQQPNGCADIQCIITQHIKQEWVAKPLVGDVILQHPVRGRIRCSVLTPRVINGWHTEASWKMKFKQEKCQVMAAGVLKNDGTILQLLIHEEPMHTWNSLKWFPPFNQICGFPFKAKHLFATNKIRQPNFVKAEFNCCGHYCVGMINCAPWLTLNAVQVCQQYEYHYCDIPKFTESMWNVDEMTVKHWQITSDPFAKCYCWAIWWHASVGYIFDLAMNPKTYQDVINNYQVWAMDGVHYFIILWRWVGVTDVTSLSFNFDTDRFSTNFWNSLRIIWTFKMLNPMCAVWEPACYFQNAYYHQFEAYQQLCIGAFKCYFVPQWLEWEDSPVSDHGLVESCKYVEDQTENLRIYKVNEMGICLNFYAKCDMRTKATQCICPCAGQAWRWYPDMMCPFGHLIPLIMGCTWIAIGWDMASAFGWKMRHEECWGAMDCKMKQAEQTNEIIWWLEHQMHPAMKSYGFPKVSPHRCTQREECNFVNRWKNISQMHWYVHQNQQGPGMLWFHEYADRAFCGSGSMGQYDHDRPYMKWFMHNLQYWAHIIECQPGMSSGHIDDPKIGTIIVMAGIGGYDQDNIDNNDKESLVWIFRLQPWRYPFVRKLPPKIWNPVDAPLSWLQGPRIMSTCDRTELNGMTTCQMPDDFPDLESMYEMYERVIMIKCDYMDDCYERNEYCYMIPAELTEFWIMHIDAYMGLQAAIDSIEDLYHEKGDTHCMAEMRSAMYGDWFGLHCPFYGLEEWGLTQKKILVDHRDLDTFTHNGAHIYWSQMIVADHFWIENKMMYFIDCKDANCHIIFVTLWNRRCVGFQNNCWYCKWCKFCLPKVDVWKKPGWNIEYEIYDEVYCVAQVWFNGTTKKIAAMAMVRSDISDEKDWPRRQDCYLTGLTLQFDQYYPKLGFKFSAFYNNMLPMFPMWEKFNNHSHETWLDDHYNGNYYRQVMFLAFNCASPYRPATEKTNRHANGNCVIDHRTRWFIVMGRQKQRVASCGSMAGDVCVNMHHCTVPAWALQAFSTKVNNGHIHQWAIETHPVWNMVPHGVLPLECCRVRQNWHTDQYNKQEEVWKFPKTKPETRPPDEDPDSCWDRKEGERGQGADGMLRETNERHWCVIRYHCAARQERANCFFHFVKVQVCPCTVIYFMPAVEKTGTFRTSDYGIFWRWCCQWCEQMFKSGIMVGSGHAWEFHMFPDAPIFCDIGSGPDPDSWWHVYEWYGHESYTPINQNVKFDQEKHYNNAWAFCYIGAYTSTFLWVELGVYRCWWKTELERKLLDEFIVVCSNSYWEWPKADDSMHYMPDDNRWTVWELKYGRWAHRAQVTIDVFTCHYGFQYANSTLEIFDDPEYTKNHASQVMIDDLGFVLCDNNRHGSTSVIYSCAQDLPNGEDKMCPWAHASYNVGYDWLIHDVYLPKKTNPIMYMVQWLQMRWEMVYPMYSDTYWNPQSRGHFWVCQDWHSYGLYCNPSYPSIGCWYMTEAFTGCIWGSFHMLHWAHKQLCEIYEPQCWSHIVCCKSPCKLMPYWGYEIDQGTADINMEYQEKAYEQCFQVQSYKIKMHAQEYEASTFDRDITSPRKCAIIFPVGVRPRGVQLLCAQQHAMTEGMWLSERLLLTAWLRHRCSRKYGHKQRFETLCRSVNFISPHEVKEAWTWYYYNNGWMDKCWEQGSGGWAWYIGEMNAHHSEWLSINYQPTCTPVCYQVDTTTPHGMPWGLQHNHVYRVQIQTTNHEVGPFQNDPWKSWTAAFDGPNCQCCETGTEFGPCTFIRMKYGYVFWKSYYKFPWWLPSAILHGAFRSFFFKCGVWWVTCKTLTSLALKFINTQPKEYKAGIEIHAYPSNYELLRCFVFYLYNRRTWCFNIWWNWHVEFESKQYMWENDYSNSQQFHSTKMKREQFKETWVFKFPSAQEDDERKPLTSDTRMDLDGQTYVAQHLGVCSPSYGNIGDDHDDMLFTHHRRFHFGGWQIPENFKPDHGKEYVNFYIALSERAERYIPKHLINAWAQDRRWTWMITWMTAGYYWKTPFCPEPEDDNNKFPSHFPHTYKQKLFYRKVYKIDLLLKMHARLAECMCCGRAINMRKGDRADHMMAMFLEWWQ'
y='FGVLMCLSDLFQYMRNSAEAYTDTAKWKFYVMYQSDQWQRHGFFPPCTNPQNSHAWSCSSRYGFWWIKTDILRMLHEWRFPVCNRRSYQPCCSSYPICCPKWNVFNIKHTVETYKMLYCHDGYNCNHRLFHSSMVKRAVLMTQLLWLEWMLGCFRSSKYKHCIYRMIQNPPPNVMSYHIMPQQVFNPNIYYYGQYAADSAISHPILVPLQHQTEPFFTHHNWDRPHYMSTNDYKEIVYRCAEDNSKATNCWRYNTIWHDAMLLMTSYSAEKSHCFVNNTQPEVAAHEMQSNVDTSRDILSTQCKDASYPEVNNQGTGNVDQGEPYQFWEREKTWKVFHGYMIITQYNMPHRHWGWHVDFFEVWEVHSIFDAVRFEYDEIMRRSWATWPQNTGFTFSFTTTWNNQTDFLKMHDDNPWLSGKQWFPEHHHSHNMNNYDCEWMVMCDKKVDVIIDLREQTTQFTPECWLADAKYFKEFWGTTRRNWIAKCYIIDGRRMMQQVERGPHMQEVCKDICGKIPPWYHFMDSKVMLARQAGEPFRVVHPPMHYARIDYDCACWGMGWNRYACITQSLMCEMEKHMKVRPSDMTNMLRYEDMVRRTWTLLGREALTSICNDIWFLKHPWMVRHWFYAVFHNDLQSAQFWIPNATHKVDVEITRHFQLAWHRICHGHQCDNKQLNRIIKKWCRTMLTIFHKWAGFKWSFRWNVHYSYKDHGAEISHVRGEVWTNLHSLMGVTGYWLPDLGQKCCLIRINVDPYGQWTHGWCRCRIPWQYMKDTGFHRLRMGDMDMKSNCPAEWQHNIIHFVLRLRDRIGNMEVNDSCQVRWEAGKSHTGSIGPACKRNKGDRFMRESTAFNGSERSNEWEVNPMDLLGWNTGHFAMVENAHDVWSACTWDSMHHHLINGQMLLACHMVFWHEKQQFIWMSHSINLTHHSLDTSKFLAAVWYWVGWMTRVLQVNVPSLLHIDYWDAAQGFASVNCIMCVHPTVGHWYNMQWVNCDVGDCSMETGFGATVQDLMRECLWRATPGFECKIMKDYQYKLWNTNYYRYMRSQAEWDMWCDLAYLRFYDFIMECCTSRKLWCCCEVDFRKIVQDRQDEEDKEFSAWCMWTNKIWCHRVIVFKLCHGGVVFNCMSHMYYTVYIPVPAPWMPIYKYGIRTYVDKIFPDQGVWQLENMNGMPILYMMPQSHNWDMKLDHCWVKNTTNRIQMAAELQTTGSSNAGLSVMPFLQMRMLEFEQSSAQQRLMVAHNRDGPMMTIKEEIRVDEWAHSAACYQRNERKQPTQHMWWSRFKNTKNMFNCNKEKEFPYGYYSEHCPRFDDSRMNWVFDYHASCMDCNYRCVIMQRWGHGHYARRHCQSFHLTSFDDIIVKQMYWWAYVNCTQRRCYVFKLKYSFAPKKYYIPVDCYQCWYPNHSVMWRTDYLENASKSCHQFWWQTQNCHLKLSFVRLHKYFHFQPFGEPPSPIGCATRIANNQYVYESKSVWFFNQKFWECQPCWWLGKPYWFQMATTHTKHIDYITTSGRSCDLMAVQSKQRLPHFGSNELQVKSMCRIQCIWPIEPYKWEAQVQGNHFIVQKGMWIWTFKGMNRCESQEDCGTYWGCGNWQNCPPFALSKYEKNEAHWSYSRLCCVRPMFFYMVRDSEGRMHSYAIVATFRRQKFFNCFSIVHGFRQQCDLWMMDAFDSYWAAFVSSVDCIGKTDNHERTEADRFAMFAKNDWKRREDRYWRYATMVGLKAWPTVRTENGHVCHQNTKKIKPWNHIHAFFWECTWTWVRCINAVYDTMPNRYNNLMPIEENVNLMPITLDDPRYQTAELYFQGQQHFDLLCKHYYHYLNPQCVQHKVQENTGYEWPQATAYHYVDTDQKKGCIITEAMQHIKQENVLKPLVWPVRGRIRISVCNGWHYKQEKCFVMAYGVCKNLQLLSHEEPMNSGHIWTWNRSGCYFLKWFPFKAKHLFATNKYRQDNFVKRAGFNCCGHYCVGMINCAPNLTLNAVQMCQQAEYHYCDIPKFTSLMWLVDEGTVKTAQACWNQIMSDPFCGRRIGVSTVGQGYIHDLAMNPKTYQDVINNDEQWSSHYFIILWRWVGVTSLSFHFDTDRFSTNFWNSLRISKMLNPMCTPPYLTNVWEPSCYSQNAYYHQFEAKEQLCIGAFKQYFTPHCQAADWRWRCDNDSPVSDHGLVDKVNFMGISLNFYAKCDMCLCAGQAWRWYPDMMCPFTYICTWIAIGWDMWSAFGWKMRHPWGRGYQTQGCKMKQAEYLNEILEHQHQPAMKSYGEDLACKLSNSPHRFTQREECNFVNRWKNISQMHQYDGMLVFHEYADCFFAARAFCGSGDKRVKMVMGQYHHDRPYMPCWMWRWIEFDVHQPWMSSGHIGCNCDEKIGTIIVYDQDNIDNNDKESLVWITRCAHVCGPWRNHHCGPPKKIWPPVDATLSWLILRWGPRICSTCGMCQMPDDFPDLMYERVIDYMDDCYERYEFYYYKAMEMYMQPAELGEFWIMHIDAYMGEQWAIDSIEDLYHEKGDTHCMSEMSAMYGDWFGQDPLRFHRCPFFQNSGGYKYFFYTCEEWTQKKILVDHRTGLIKNIFNFGQIMVDTFYHNGASHFWIENKMMYFIDCKDANCHICVHWVQMVRNVTLWNVNRCVGFRNNCWYHLVAWIKWCKFWLPKWKKPGWNPEYEINDEDTGTSIYCEWWYVAQVNGTTKKCIAMVFVYANEGMVRSDIDCYLTCHTLILTLQFDQYYPKLGFKFSAFYNNMLPMFPMWEKFNNHSHSTWLADHYNGPLFDAGGYTRQVMFTDTMRAWFNCASPYRPATEKTERHANGRCVIRTRWHIVMGRQKQRVHRCSEWVTKYTGSMAGFVCVVPAWAQPTNSNGIHRCGHEPFKPVWNMVEWTYRSNTQKNVLECCRVRQNWHTDVWETRKPDEDIKGMHNIDQSCWDRCEGERGHTAQFQYIGAVGMLRETNERKWCVSPLAIFICERYHCPIQGSVEHARQERANCGFHFVKVKVCPCTVIYFMPAVEKTGTFRPRRAQCSDYGRCRWRWCCQWCKLMFKSNQWVFRGIPVGSGHAWEPHMFPDAPIFPLSWFLTEWWGSCRYYHWVSYTDMCWYKLYLCVTNNAWSWNYIGAYTSTFLWVELGVYRCWWKTDEFIVVCSNSYWEHPKADDSMHYMMKCAYDDHRWTVWELKYGRIEKYRRDDKNIDVTKTCHYGFQYANSTLGADEIFDDHEYTKNHALQMMIDDGGFTLSCAQDLPNGEDKMCPWAHASMEVPSIVSDWLIHDVYLPYFMQWLMQSHKQMIWEMVYPMYSDTYWNPQSRGHFWWCQDWHSYGVCGLYCNPSYPSINIETCQRTNTQYMTEAFTGCIWGAHKQLCEIYDPQCWSHINITIPPTCKSPCKLMRWNNDGIKNMCPWLIWLGGKGTADINMMYKAYWCRQVQSYKLKMQAQEYCASTFDRAITSPRPGAIIFPVGLCAMTEGMWLSEVLLLTGWLRHRCIRIYGHKQRFETLCRSVNFISPMEVVPVVVIFGLAWTWYYYNNLWMNSPQWYWETHMYWTANGSRGWSWYIGEMNWLSINYQPTCTFPYHSFVYQVATTTPHGMPWGLQHNIVYRVQIQTSNHPVGPFQNDPWWTAAFDGPNCQCCETGTEFGPCTAFYQSIRMKYGGFACLWTIHVFPLPSAILHGFFFKCKIVEHWTPMNWWDKTLTSSACKFINRSWHGRDQPKEYKAGIETHVKWSNYELLCCFVFYYYNRKTWCFNIWWNWHVEFESKYYEKYFYMGEENGWVMDNSNSQGFHQQETWVFKFPSAQEDDERKPLTSDTRMDLDDSGMAQTREMAVAQNLIGDIHDDVLFTVHRRFHFGGWQIPENNRQLQWQCAETAEIPKTAQDRRRTIAMCKQMIPWMNAGYYWKTPFCPECEWNDNCCSSHFPKPGWSHGWWGLLAECMCCGRAINMTKGDHAMAMFLEWWQ'

alScore, LCS, a,b,_ = globalAlignment(x,y)

print(alScore)
print(a)
print(b)


CPU times: user 9 µs, sys: 20 µs, total: 29 µs
Wall time: 81.3 µs


KeyboardInterrupt: 

In [148]:
x="CCAATCTACTACTGCTTGCA"
y="GCCACTCTCGCTGTACTGTG"
lcsq(x,y,indel=0)

('CCATCTCTACTGTG',
 array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0],
        [ 0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1],
        [ 0,  0,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2,  2,  2],
        [ 0,  0,  1,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3],
        [ 0,  0,  1,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,
          4,  4,  4,  4,  4],
        [ 0,  0,  1,  2,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  5,  5,  5,  5],
        [ 0,  0,  1,  2,  3,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,
          5,  5,  5,  5,  5],
        [ 0,  0,  1,  2,  3,  4,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,
          6,  6,  6,  6,  6],
        [ 0,  0,  1,  2,  3,  4,  5,  5,  6,  6,  6,  6,  6,  6,  6,  7,
          7,  7,  7,  7,  7],
        [ 0,  0,  1,  2,  3,  4,  5,  6,  6,  7,  7,  

In [147]:
def lcsq(x, y,indel=-5):
    m = len(x)
    n = len(y)
    table = np.zeros((m+1, n+1), dtype=int)
    for i in range(1, m+1):
        for j in range(1, n+1):
            if x[i-1] == y[j-1]:
                table[i, j] = table[i-1, j-1] + scorematch(x[i-1],y[j-1])
            else:
                table[i, j] = max(table[i, j-1] + indel, table[i-1, j] + indel)
    
    i, j = m, n
    w = ''
    while (i > 0) and (j > 0):
        if x[i-1] == y[j-1]:
            w = x[i-1] + w
            i -= 1
            j -= 1
        else:
            if table[i, j-1] > table[i-1, j]:
                j -= 1
            else:
                i -= 1
    return w,table

In [226]:
x='PLEASANTLY'
y='MEANLY'
lcsq(x, y)

('EANLY',
 array([[  0,   0,   0,   0,   0,   0,   0],
        [  0,  -5,  -5,  -5,  -5,  -5,  -5],
        [  0,  -5, -10, -10, -10,  -1,  -6],
        [  0,  -5,   0,  -5, -10,  -6, -11],
        [  0,  -5,  -5,   4,  -1,  -6, -11],
        [  0,  -5, -10,  -1,  -6, -11, -16],
        [  0,  -5, -10,  -6, -11, -16, -21],
        [  0,  -5, -10, -11,   0,  -5, -10],
        [  0,  -5, -10, -15,  -5, -10, -15],
        [  0,  -5, -10, -15, -10,  -1,  -6],
        [  0,  -5, -10, -15, -15,  -6,   6]]))

In [167]:
x='ILYPRQSMICMSFCFWDMWKKDVPVVLMMFLERRQMQSVFSWLVTVKTDCGKGIYNHRKYLGLPTMTAGDWHWIKKQNDPHEWFQGRLETAWLHSTFLYWKYFECDAVKVCMDTFGLFGHCDWDQQIHTCTHENEPAIAFLDLYCRHSPMCDKLYPVWDMACQTCHFHHSWFCRNQEMWMKGDVDDWQWGYHYHTINSAQCNQWFKEICKDMGWDSVFPPRHNCQRHKKCMPALYAGIWMATDHACTFMVRLIYTENIAEWHQVYCYRSMNMFTCGNVCLRCKSWIFVKNYMMAPVVNDPMIEAFYKRCCILGKAWYDMWGICPVERKSHWEIYAKDLLSFESCCSQKKQNCYTDNWGLEYRLFFQSIQMNTDPHYCQTHVCWISAMFPIYSPFYTSGPKEFYMWLQARIDQNMHGHANHYVTSGNWDSVYTPEKRAGVFPVVVPVWYPPQMCNDYIKLTYECERFHVEGTFGCNRWDLGCRRYIIFQCPYCDTMKICYVDQWRSIKEGQFRMSGYPNHGYWFVHDDHTNEWCNQPVLAKFVRSKIVAICKKSQTVFHYAYTPGYNATWPQTNVCERMYGPHDNLLNNQQNVTFWWKMVPNCGMQILISCHNKMKWPTSHYVFMRLKCMHVLMQMEYLDHFTGPGEGDFCRNMQPYMHQDLHWEGSMRAILEYQAEHHRRAFRAELCAQYDQEIILWSGGWGVQDCGFHANYDGSLQVVSGEPCSMWCTTVMQYYADCWEKCMFA'
y='ILIPRQQMGCFPFPWHFDFCFWSAHHSLVVPLNPQMQTVFQNRGLDRVTVKTDCHDHRWKWIYNLGLPTMTAGDWHFIKKHVVRANNPHQWFQGRLTTAWLHSTFLYKKTEYCLVRHSNCCHCDWDQIIHTCAFIAFLDLYQRHWPMCDKLYCHFHHSWFCRNQEMSMDWNQWFPWDSVPRANCLEEGALIALYAGIWANSMKRDMKTDHACTVRLIYVCELHAWLKYCYTSINMLCGNVCLRCKSWIFVKLFYMYAPVVNTIEANSPHYYKRCCILGQGICPVERKSHCEIYAKDLLSFESCCSQKQNCYTDNWGLEYRLFFQHIQMECTDPHANRGWTSCQTAKYWHFNLDDRPPKEFYMWLQATPTDLCMYQHCLMFKIVKQNFRKQHGHANPAASTSGNWDSVYTPEKMAYKDWYVSHPPVDMRRNGSKMVPVWYPPGIWHWKQSYKLTYECFFTVPGRFHVEGTFGCNRWDHQPGTRRDRQANHQFQCPYSDTMAIWEHAYTYVDQWRSIKEGQMPMSGYPNHGQWNVHDDHTNEQERSPICNQPVLAKFVRSKNVSNHEICKKSQTVFHWACEAQTNVCERMLNNQHVAVKRNVTFWWQMVPNCLWSCHNKMTWPTRPEQHRLFFVKMRLKCMHEYLDVAPSDFCRNMQAYMHSMRAILEYQADFDLKRRLRAIAPMDLCAQYDQEIILWSGGYIYDQSLQVVSCEGCSYYADCYVKCINVKEKCMFA'
lcsq(x, y)

('ILPRQMCFFWDWVPLQMQVFLVTVKTDCHRKYLGLPTMTAGDWHIKKNPHWFQGRLTAWLHSTFLYKYCVCHCDWDQIHTCAIAFLDLYRHPMCDKLYCHFHHSWFCRNQEMMDWNQWFWDSVPRNCALYAGIWMTDHACTVRLIYEAWYCYSNMCGNVCLRCKSWIFVKYMAPVVNIEAYKRCCILGGICPVERKSHEIYAKDLLSFESCCSQKQNCYTDNWGLEYRLFFQIQMTDPHCQTWFPPKEFYMWLQADQNHGHANTSGNWDSVYTPEKAVPVVPVWYPPQYKLTYECRFHVEGTFGCNRWDGRRFQCPYDTMIYVDQWRSIKEGQMSGYPNHGWVHDDHTNECNQPVLAKFVRSKVICKKSQTVFHAAQTNVCERMLNNQNVTFWWMVPNCLSCHNKMWPTHFMRLKCMHEYLDPDFCRNMQYMHSMRAILEYQARRRALCAQYDQEIILWSGGYDSLQVVSGCSYYADCEKCMFA',
 array([[  0,   0,   0, ...,   0,   0,   0],
        [  0,   4,  -1, ...,  -5,  -5,  -5],
        [  0,  -1,   8, ..., -10, -10, -10],
        ...,
        [  0,  -5, -10, ..., 293, 288, 283],
        [  0,  -5, -10, ..., 288, 299, 294],
        [  0,  -5, -10, ..., 283, 294, 303]]))

In [176]:
%time 
x='ILYPRQSMICMSFCFWDMWKKDVPVVLMMFLERRQMQSVFSWLVTVKTDCGKGIYNHRKYLGLPTMTAGDWHWIKKQNDPHEWFQGRLETAWLHSTFLYWKYFECDAVKVCMDTFGLFGHCDWDQQIHTCTHENEPAIAFLDLYCRHSPMCDKLYPVWDMACQTCHFHHSWFCRNQEMWMKGDVDDWQWGYHYHTINSAQCNQWFKEICKDMGWDSVFPPRHNCQRHKKCMPALYAGIWMATDHACTFMVRLIYTENIAEWHQVYCYRSMNMFTCGNVCLRCKSWIFVKNYMMAPVVNDPMIEAFYKRCCILGKAWYDMWGICPVERKSHWEIYAKDLLSFESCCSQKKQNCYTDNWGLEYRLFFQSIQMNTDPHYCQTHVCWISAMFPIYSPFYTSGPKEFYMWLQARIDQNMHGHANHYVTSGNWDSVYTPEKRAGVFPVVVPVWYPPQMCNDYIKLTYECERFHVEGTFGCNRWDLGCRRYIIFQCPYCDTMKICYVDQWRSIKEGQFRMSGYPNHGYWFVHDDHTNEWCNQPVLAKFVRSKIVAICKKSQTVFHYAYTPGYNATWPQTNVCERMYGPHDNLLNNQQNVTFWWKMVPNCGMQILISCHNKMKWPTSHYVFMRLKCMHVLMQMEYLDHFTGPGEGDFCRNMQPYMHQDLHWEGSMRAILEYQAEHHRRAFRAELCAQYDQEIILWSGGWGVQDCGFHANYDGSLQVVSGEPCSMWCTTVMQYYADCWEKCMFA'
y='ILIPRQQMGCFPFPWHFDFCFWSAHHSLVVPLNPQMQTVFQNRGLDRVTVKTDCHDHRWKWIYNLGLPTMTAGDWHFIKKHVVRANNPHQWFQGRLTTAWLHSTFLYKKTEYCLVRHSNCCHCDWDQIIHTCAFIAFLDLYQRHWPMCDKLYCHFHHSWFCRNQEMSMDWNQWFPWDSVPRANCLEEGALIALYAGIWANSMKRDMKTDHACTVRLIYVCELHAWLKYCYTSINMLCGNVCLRCKSWIFVKLFYMYAPVVNTIEANSPHYYKRCCILGQGICPVERKSHCEIYAKDLLSFESCCSQKQNCYTDNWGLEYRLFFQHIQMECTDPHANRGWTSCQTAKYWHFNLDDRPPKEFYMWLQATPTDLCMYQHCLMFKIVKQNFRKQHGHANPAASTSGNWDSVYTPEKMAYKDWYVSHPPVDMRRNGSKMVPVWYPPGIWHWKQSYKLTYECFFTVPGRFHVEGTFGCNRWDHQPGTRRDRQANHQFQCPYSDTMAIWEHAYTYVDQWRSIKEGQMPMSGYPNHGQWNVHDDHTNEQERSPICNQPVLAKFVRSKNVSNHEICKKSQTVFHWACEAQTNVCERMLNNQHVAVKRNVTFWWQMVPNCLWSCHNKMTWPTRPEQHRLFFVKMRLKCMHEYLDVAPSDFCRNMQAYMHSMRAILEYQADFDLKRRLRAIAPMDLCAQYDQEIILWSGGYIYDQSLQVVSCEGCSYYADCYVKCINVKEKCMFA'
lcsq(x, y)

CPU times: user 14 µs, sys: 28 µs, total: 42 µs
Wall time: 2.96 ms


('ILPRQMCFFWDWVPLQMQVFLVTVKTDCHRKYLGLPTMTAGDWHIKKNPHWFQGRLTAWLHSTFLYKYCVCHCDWDQIHTCAIAFLDLYRHPMCDKLYCHFHHSWFCRNQEMMDWNQWFWDSVPRNCALYAGIWMTDHACTVRLIYEAWYCYSNMCGNVCLRCKSWIFVKYMAPVVNIEAYKRCCILGGICPVERKSHEIYAKDLLSFESCCSQKQNCYTDNWGLEYRLFFQIQMTDPHCQTWFPPKEFYMWLQADQNHGHANTSGNWDSVYTPEKAVPVVPVWYPPQYKLTYECRFHVEGTFGCNRWDGRRFQCPYDTMIYVDQWRSIKEGQMSGYPNHGWVHDDHTNECNQPVLAKFVRSKVICKKSQTVFHAAQTNVCERMLNNQNVTFWWMVPNCLSCHNKMWPTHFMRLKCMHEYLDPDFCRNMQYMHSMRAILEYQARRRALCAQYDQEIILWSGGYDSLQVVSGCSYYADCEKCMFA',
 array([[  0,   0,   0, ...,   0,   0,   0],
        [  0,   4,  -1, ...,  -5,  -5,  -5],
        [  0,  -1,   8, ..., -10, -10, -10],
        ...,
        [  0,  -5, -10, ..., 293, 288, 283],
        [  0,  -5, -10, ..., 288, 299, 294],
        [  0,  -5, -10, ..., 283, 294, 303]]))

In [ ]:
m = len(x)
n = len(y)
table = np.zeros((m+2, n+2), dtype=int)
for i in range(1, m+1):
    for j in range(1, n+1):
        if x[i-1] == y[j-1]:
            table[i, j] = table[i-1, j-1] + scorematch(x[i-1],y[j-1])
        else:
            table[i, j] = max(table[i, j-1] + indel, table[i-1, j] + indel)

In [228]:

table = np.zeros((m+1, n+1), dtype=int)
for _ in range(1,table.shape[1]):
    table[0][_] = table[0][_-1] + indel

for _ in range(1,table.shape[0]):
    table[_][0] = table[_-1][0] + indel
table

array([[  0,  -5, -10, -15, -20, -25, -30],
       [ -5,   0,   0,   0,   0,   0,   0],
       [-10,   0,   0,   0,   0,   0,   0],
       [-15,   0,   0,   0,   0,   0,   0],
       [-20,   0,   0,   0,   0,   0,   0],
       [-25,   0,   0,   0,   0,   0,   0],
       [-30,   0,   0,   0,   0,   0,   0],
       [-35,   0,   0,   0,   0,   0,   0],
       [-40,   0,   0,   0,   0,   0,   0],
       [-45,   0,   0,   0,   0,   0,   0],
       [-50,   0,   0,   0,   0,   0,   0]])

In [232]:

for i in range(1, m+1):
    for j in range(1, n+1):
        if x[i-1] == y[j-1]:
            table[i, j] = table[i-1, j-1] + scorematch(x[i-1],y[j-1])
        else:
            table[i, j] = max(table[i, j-1] + indel,
                              table[i-1, j] + indel,
                              table[i-1, j-1] + scorematch(x[i-1],y[j-1]))
table

array([[  0,  -5, -10, -15, -20, -25, -30],
       [ -5,  -2,  -6, -11, -16, -21, -26],
       [-10,  -3,  -5,  -7, -12, -12, -17],
       [-15,  -8,   2,  -3,  -7, -12, -14],
       [-20, -13,  -3,   6,   1,  -4,  -9],
       [-25, -18,  -8,   1,   7,   2,  -3],
       [-30, -23, -13,  -4,   2,   6,   1],
       [-35, -28, -18,  -9,   2,   1,   4],
       [-40, -33, -23, -14,  -3,   1,  -1],
       [-45, -38, -28, -19,  -8,   1,   0],
       [-50, -43, -33, -24, -13,  -4,   8]])

In [237]:
i = table.shape[0]
j = table.shape[1]
(i,j)

(11, 7)

In [258]:
def globAlign(x,y):
    m = len(x)
    n = len(y)
    table = np.zeros((m+2, n+2), dtype=int)
    for i in range(1, m+1):
        for j in range(1, n+1):
            if x[i-1] == y[j-1]:
                table[i, j] = table[i-1, j-1] + scorematch(x[i-1],y[j-1])
            else:
                table[i, j] = max(table[i, j-1] + indel, table[i-1, j] + indel)
    table = np.zeros((m+1, n+1), dtype=int)
    for _ in range(1,table.shape[1]):
        table[0][_] = table[0][_-1] + indel

    for _ in range(1,table.shape[0]):
        table[_][0] = table[_-1][0] + indel
    table
    for i in range(1, m+1):
        for j in range(1, n+1):
            if x[i-1] == y[j-1]:
                table[i, j] = table[i-1, j-1] + scorematch(x[i-1],y[j-1])
            else:
                table[i, j] = max(table[i, j-1] + indel,
                                  table[i-1, j] + indel,
                                  table[i-1, j-1] + scorematch(x[i-1],y[j-1]))

    i = table.shape[0] - 1
    j = table.shape[1] - 1

    a = ''
    b = ''
    lcs = ''
    while (j>0 or i>0):
        if x[i-1] == y[j-1]:
            #print(x[i-1],y[j-1],Fmatrix[i,j])
            lcs += x[i-1]
            j -=1
            i -=1
            a +=x[i]
            b +=y[j]
            paths.append([(np.inf),(x[i],y[j]) ])
        else:
            diag = Fmatrix[i-1,j-1] + scorematch(x[i-1],y[j-1])
            left = Fmatrix[i-1,j] + indel
            up = Fmatrix[i,j-1] + indel
            maxCell = np.argmax([diag,left,up])
            if maxCell == 0:
                j -=1
                i -=1
                a +=x[i]
                b +=y[j]
            else:
                if maxCell == 1:
                    i -= 1
                    a += x[i]
                    b += '-'
                else:
                    j -= 1
                    a += '-'
                    b += y[j]
    #####
    maxScore = table[table.shape[0] - 1,table.shape[1] - 1]                
    return lcs, b[::-1], a[::-1], maxScore, table

In [264]:
def globAlign2(x,y):
    m = len(x)
    n = len(y)
    table = np.zeros((m+2, n+2), dtype=int)
    for i in range(1, m+1):
        for j in range(1, n+1):
            if x[i-1] == y[j-1]:
                table[i, j] = table[i-1, j-1] + scorematch(x[i-1],y[j-1])
            else:
                table[i, j] = max(table[i, j-1] + indel, table[i-1, j] + indel)
    table = np.zeros((m+1, n+1), dtype=int)
#     for _ in range(1,table.shape[1]):
#         table[0][_] = table[0][_-1] + indel

#     for _ in range(1,table.shape[0]):
#         table[_][0] = table[_-1][0] + indel
    table
    for i in range(1, m+1):
        for j in range(1, n+1):
            if x[i-1] == y[j-1]:
                table[i, j] = table[i-1, j-1] + scorematch(x[i-1],y[j-1])
            else:
                table[i, j] = max(table[i, j-1] + indel,
                                  table[i-1, j] + indel,
                                  table[i-1, j-1] + scorematch(x[i-1],y[j-1]))

    i = table.shape[0] - 1
    j = table.shape[1] - 1

    a = ''
    b = ''
    lcs = ''
    while (j>0 or i>0):
        if x[i-1] == y[j-1]:
            #print(x[i-1],y[j-1],Fmatrix[i,j])
            lcs += x[i-1]
            j -=1
            i -=1
            a +=x[i]
            b +=y[j]
            paths.append([(np.inf),(x[i],y[j]) ])
        else:
            diag = Fmatrix[i-1,j-1] + scorematch(x[i-1],y[j-1])
            left = Fmatrix[i-1,j] + indel
            up = Fmatrix[i,j-1] + indel
            maxCell = np.argmax([diag,left,up])
            if maxCell == 0:
                j -=1
                i -=1
                a +=x[i]
                b +=y[j]
            else:
                if maxCell == 1:
                    i -= 1
                    a += x[i]
                    b += '-'
                else:
                    j -= 1
                    a += '-'
                    b += y[j]
    #####
    maxScore = table[table.shape[0] - 1,table.shape[1] - 1]                
    return lcs, b[::-1], a[::-1], maxScore, table

In [262]:
%time
lcs, b, a, maxScore, table = globAlign(x,y)
print(maxScore)
print(a)
print(b)

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 11.7 µs
8
PLEASANTLY
-ME--AN-LY


In [263]:
%time
x='ILYPRQSMICMSFCFWDMWKKDVPVVLMMFLERRQMQSVFSWLVTVKTDCGKGIYNHRKYLGLPTMTAGDWHWIKKQNDPHEWFQGRLETAWLHSTFLYWKYFECDAVKVCMDTFGLFGHCDWDQQIHTCTHENEPAIAFLDLYCRHSPMCDKLYPVWDMACQTCHFHHSWFCRNQEMWMKGDVDDWQWGYHYHTINSAQCNQWFKEICKDMGWDSVFPPRHNCQRHKKCMPALYAGIWMATDHACTFMVRLIYTENIAEWHQVYCYRSMNMFTCGNVCLRCKSWIFVKNYMMAPVVNDPMIEAFYKRCCILGKAWYDMWGICPVERKSHWEIYAKDLLSFESCCSQKKQNCYTDNWGLEYRLFFQSIQMNTDPHYCQTHVCWISAMFPIYSPFYTSGPKEFYMWLQARIDQNMHGHANHYVTSGNWDSVYTPEKRAGVFPVVVPVWYPPQMCNDYIKLTYECERFHVEGTFGCNRWDLGCRRYIIFQCPYCDTMKICYVDQWRSIKEGQFRMSGYPNHGYWFVHDDHTNEWCNQPVLAKFVRSKIVAICKKSQTVFHYAYTPGYNATWPQTNVCERMYGPHDNLLNNQQNVTFWWKMVPNCGMQILISCHNKMKWPTSHYVFMRLKCMHVLMQMEYLDHFTGPGEGDFCRNMQPYMHQDLHWEGSMRAILEYQAEHHRRAFRAELCAQYDQEIILWSGGWGVQDCGFHANYDGSLQVVSGEPCSMWCTTVMQYYADCWEKCMFA'
y='ILIPRQQMGCFPFPWHFDFCFWSAHHSLVVPLNPQMQTVFQNRGLDRVTVKTDCHDHRWKWIYNLGLPTMTAGDWHFIKKHVVRANNPHQWFQGRLTTAWLHSTFLYKKTEYCLVRHSNCCHCDWDQIIHTCAFIAFLDLYQRHWPMCDKLYCHFHHSWFCRNQEMSMDWNQWFPWDSVPRANCLEEGALIALYAGIWANSMKRDMKTDHACTVRLIYVCELHAWLKYCYTSINMLCGNVCLRCKSWIFVKLFYMYAPVVNTIEANSPHYYKRCCILGQGICPVERKSHCEIYAKDLLSFESCCSQKQNCYTDNWGLEYRLFFQHIQMECTDPHANRGWTSCQTAKYWHFNLDDRPPKEFYMWLQATPTDLCMYQHCLMFKIVKQNFRKQHGHANPAASTSGNWDSVYTPEKMAYKDWYVSHPPVDMRRNGSKMVPVWYPPGIWHWKQSYKLTYECFFTVPGRFHVEGTFGCNRWDHQPGTRRDRQANHQFQCPYSDTMAIWEHAYTYVDQWRSIKEGQMPMSGYPNHGQWNVHDDHTNEQERSPICNQPVLAKFVRSKNVSNHEICKKSQTVFHWACEAQTNVCERMLNNQHVAVKRNVTFWWQMVPNCLWSCHNKMTWPTRPEQHRLFFVKMRLKCMHEYLDVAPSDFCRNMQAYMHSMRAILEYQADFDLKRRLRAIAPMDLCAQYDQEIILWSGGYIYDQSLQVVSCEGCSYYADCYVKCINVKEKCMFA'
lcs, b, a, maxScore, table = globAlign(x,y)
print(maxScore)
print(a)
print(b)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.96 µs


IndexError: index 738 is out of bounds for axis 0 with size 11

In [272]:
%time
# x='ILYPRQSMICMSFCFWDMWKKDVPVVLMMFLERRQMQSVFSWLVTVKTDCGKGIYNHRKYLGLPTMTAGDWHWIKKQNDPHEWFQGRLETAWLHSTFLYWKYFECDAVKVCMDTFGLFGHCDWDQQIHTCTHENEPAIAFLDLYCRHSPMCDKLYPVWDMACQTCHFHHSWFCRNQEMWMKGDVDDWQWGYHYHTINSAQCNQWFKEICKDMGWDSVFPPRHNCQRHKKCMPALYAGIWMATDHACTFMVRLIYTENIAEWHQVYCYRSMNMFTCGNVCLRCKSWIFVKNYMMAPVVNDPMIEAFYKRCCILGKAWYDMWGICPVERKSHWEIYAKDLLSFESCCSQKKQNCYTDNWGLEYRLFFQSIQMNTDPHYCQTHVCWISAMFPIYSPFYTSGPKEFYMWLQARIDQNMHGHANHYVTSGNWDSVYTPEKRAGVFPVVVPVWYPPQMCNDYIKLTYECERFHVEGTFGCNRWDLGCRRYIIFQCPYCDTMKICYVDQWRSIKEGQFRMSGYPNHGYWFVHDDHTNEWCNQPVLAKFVRSKIVAICKKSQTVFHYAYTPGYNATWPQTNVCERMYGPHDNLLNNQQNVTFWWKMVPNCGMQILISCHNKMKWPTSHYVFMRLKCMHVLMQMEYLDHFTGPGEGDFCRNMQPYMHQDLHWEGSMRAILEYQAEHHRRAFRAELCAQYDQEIILWSGGWGVQDCGFHANYDGSLQVVSGEPCSMWCTTVMQYYADCWEKCMFA'
# y='ILIPRQQMGCFPFPWHFDFCFWSAHHSLVVPLNPQMQTVFQNRGLDRVTVKTDCHDHRWKWIYNLGLPTMTAGDWHFIKKHVVRANNPHQWFQGRLTTAWLHSTFLYKKTEYCLVRHSNCCHCDWDQIIHTCAFIAFLDLYQRHWPMCDKLYCHFHHSWFCRNQEMSMDWNQWFPWDSVPRANCLEEGALIALYAGIWANSMKRDMKTDHACTVRLIYVCELHAWLKYCYTSINMLCGNVCLRCKSWIFVKLFYMYAPVVNTIEANSPHYYKRCCILGQGICPVERKSHCEIYAKDLLSFESCCSQKQNCYTDNWGLEYRLFFQHIQMECTDPHANRGWTSCQTAKYWHFNLDDRPPKEFYMWLQATPTDLCMYQHCLMFKIVKQNFRKQHGHANPAASTSGNWDSVYTPEKMAYKDWYVSHPPVDMRRNGSKMVPVWYPPGIWHWKQSYKLTYECFFTVPGRFHVEGTFGCNRWDHQPGTRRDRQANHQFQCPYSDTMAIWEHAYTYVDQWRSIKEGQMPMSGYPNHGQWNVHDDHTNEQERSPICNQPVLAKFVRSKNVSNHEICKKSQTVFHWACEAQTNVCERMLNNQHVAVKRNVTFWWQMVPNCLWSCHNKMTWPTRPEQHRLFFVKMRLKCMHEYLDVAPSDFCRNMQAYMHSMRAILEYQADFDLKRRLRAIAPMDLCAQYDQEIILWSGGYIYDQSLQVVSCEGCSYYADCYVKCINVKEKCMFA'
w='ILYPRQSM'
v='LIPRQQMG'
lcs, b, a, maxScore, table = globAlign(w,v)
print(maxScore)
print(a)
print(b)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


IndexError: index 7 is out of bounds for axis 1 with size 7

In [275]:
x ,y = w,v
m = len(x)
n = len(y)
table = np.zeros((m+2, n+2), dtype=int)
for i in range(1, m+1):
    for j in range(1, n+1):
        if x[i-1] == y[j-1]:
            table[i, j] = table[i-1, j-1] + scorematch(x[i-1],y[j-1])
        else:
            table[i, j] = max(table[i, j-1] + indel, table[i-1, j] + indel)
table = np.zeros((m+1, n+1), dtype=int)
for _ in range(1,table.shape[1]):
    table[0][_] = table[0][_-1] + indel

for _ in range(1,table.shape[0]):
    table[_][0] = table[_-1][0] + indel
table
for i in range(1, m+1):
    for j in range(1, n+1):
        if x[i-1] == y[j-1]:
            table[i, j] = table[i-1, j-1] + scorematch(x[i-1],y[j-1])
        else:
            table[i, j] = max(table[i, j-1] + indel,
                              table[i-1, j] + indel,
                              table[i-1, j-1] + scorematch(x[i-1],y[j-1]))
table

array([[  0,  -5, -10, -15, -20, -25, -30, -35, -40],
       [ -5,   2,  -1,  -6, -11, -16, -21, -26, -31],
       [-10,  -1,   4,  -1,  -6, -11, -16, -19, -24],
       [-15,  -6,  -1,   1,  -3,  -7, -12, -17, -22],
       [-20, -11,  -6,   6,   1,  -4,  -8, -13, -18],
       [-25, -16, -11,   1,  11,   6,   1,  -4,  -9],
       [-30, -21, -16,  -4,   6,  16,  11,   6,   1],
       [-35, -26, -21,  -9,   1,  11,  16,  11,   6],
       [-40, -31, -25, -14,  -4,   6,  11,  21,  16]])

In [280]:
i = table.shape[0] - 1
j = table.shape[1] - 1
a = ''
b = ''
lcs = ''

In [279]:

while (j>0 or i>0):
    print(i,j)
    if x[i-1] == y[j-1]:
        print(x[i-1],y[j-1],Fmatrix[i,j])
        lcs += x[i-1]
        j -=1
        i -=1
        a +=x[i]
        b +=y[j]
        paths.append([(np.inf),(x[i],y[j]) ])
    else:
        diag = Fmatrix[i-1,j-1] + scorematch(x[i-1],y[j-1])
        left = Fmatrix[i-1,j] + indel
        up = Fmatrix[i,j-1] + indel
        maxCell = np.argmax([diag,left,up])
        if maxCell == 0:
            j -=1
            i -=1
            a +=x[i]
            b +=y[j]
        else:
            if maxCell == 1:
                i -= 1
                a += x[i]
                b += '-'
            else:
                j -= 1
                a += '-'
                b += y[j]
#####
maxScore = table[table.shape[0] - 1,table.shape[1] - 1]                
return lcs, b[::-1], a[::-1], maxScore, table

8 8


IndexError: index 7 is out of bounds for axis 1 with size 7

In [16]:
%time
x='FGVLMCLSDLFQYQRNSAEAYTDTAKWAYQIYQIVRHGFFPPCTNPSHAWYKRYYHVEYTEGNIQDITRWTHEWRRSYQPMMRAQPMHWAECTHRCCSSYPICCPKWNVFNIKTLYKMLYCHDGYNCNHRLVVRSLEHGCFRFLLKNNDVERTKHCIYGMIYNPPPNVMSYIPRGMCNSYHSMIQQSLVSTELHQGDQFVFNPNIYYYGMVFKLLYQFMSGPILVPMPTICTDHYQMSCRCADMNSICWRAHDPQAEQGPNKSHCDSMKFVNNYAHEPDRDILSTQCKDASYPERNNQGTGNVDQGCQKKNEPYQLKTWKVFHTMRGYGITQYNMPHRMHFEVVNWYWEVHSIFDAVRFEYYEIMRRSWWFTFSFTQSWNNCTDFLKLYQTWCRLGGLHHHMHENNYDCEWMVMCDKKVDVIIDLREPLRICWLADAKYFKEDNKRCYSHKAHHIAKCNFQVERGEGHPALNAWYGFMDSKVMLFRQAGEPFRVMLVHPDMHYARIDYDCACWGMGWNRYGCITQSKMCESEKHFEAHQMNRPTIKCRKNMLRYEMVRRTWTLLGREINVFLHVYCYSHAWHRIGKVFWAQFHVDTITRHTLLAWTMLHTIIHRIWHGHQFDNKQLNRIIKKSCRTMLTIFGMRNEFHKWAGFKWSQYRWNVIMHAHYSYKDHGAEISHVRWVTTFGSAGEVFTNPGLNYWLPDDLGCNVDPYHHGSSVAMAHRACQNMTTRLGNVEKRMGDMDMKSNLDQRDRIGNMEVNAVRWMAGKSHTSRACKREAGEWTSFEWEVDQYVHKGYNPMDLWKGRKPSLGWNTGHFAMVENEHDVWIACVWDSMHHNADKRGFKLFFTNRLMHVWDECKQYRYFSNKQQFIWMSLSINLTSHSLDISKSYWVFKHGWMTRVLVCNQNYQQECMPDNDPMSNHPIDYRFAPVHCIMCVHNQYTHPTVGHWYNMQWSWKVNCDVGDCSYWSVKAMETGFGATVQCDLADQDRRATPGSKTMCECPKLWNTNYYRYMRSQDEWDMWCDLAYLRFYDSSHYFRKIFQDRQDEEWKEFSAWCMWQRVIVFKLCIDFCIMQREMGGVTMYYTVDAPVGIRTYVDWEFIATRAAWNLENDILYMMPQGHNWDMDHCWVKGNRIQMAACVCQHIAELQTTIQGSSNHPFLQGRMLEFEQSSAQQRLTAFNMDGYVQYILVKYRWITCACTQPTQHMWWSRFKNPTKNMFNCNKEKEFPKGYYSEGPRFDDSRMNWSFDYHANQVIMQRWGHDSGKSRLKVRHCQSFHLTSFDDIINDQAVKQMYWWAYVKCTQEDEHKQVDWCPLKYSFVCRVAPKCYYIPVDCYQDAWYPNPQRNSSFSHSVMWRTDYNENRSKSFWQILQNPKLSFVRLEKYFHFQPFGEPPSPIGCATCIYPNHYAERANNQCVYESKSVSFFNQMCYCKRMLFPLGWWAWMGWDWWHLYIKKPVCEIDQILLEPWISNLEPSGRANLMAVIGCWAHLHRSEWRDQRLYHFKSCEQRKMPQPDCMCRIQCIWPIVVQGNHFIVQKGMWIMFKGMNRCESQLNRSYSHDVIEGNWQWCPFFALSKYEKNMAHWKNYKTLSRLCVYNVRCSEGRMHVPPMCKPPIWPPFRRYPVSYAIVATFRRQWYGTNVLSWVHGFRQQCDLWMMDAMSYWAAFWCGWSSVDCFGKTDNHASWTRGGERTEAVCFAMFAFVDWRNKIDRATEDRYTIVFVWRYWMVGLKNENGHKCHQNTKKIKRLFWECMLWTMLLTWTDRCINAVSRYQPIELNVNLCPITLDCDPRYQTYWQLNETELYFLAQQHFDLLCKFYYCYLNPQCVQHKVQENTTQCILFDYEWPQAQQPNGCADIQCIITQHIKQEWVAKPLVGDVILQHPVRGRIRCSVLTPRVINGWHTEASWKMKFKQEKCQVMAAGVLKNDGTILQLLIHEEPMHTWNSLKWFPPFNQICGFPFKAKHLFATNKIRQPNFVKAEFNCCGHYCVGMINCAPWLTLNAVQVCQQYEYHYCDIPKFTESMWNVDEMTVKHWQITSDPFAKCYCWAIWWHASVGYIFDLAMNPKTYQDVINNYQVWAMDGVHYFIILWRWVGVTDVTSLSFNFDTDRFSTNFWNSLRIIWTFKMLNPMCAVWEPACYFQNAYYHQFEAYQQLCIGAFKCYFVPQWLEWEDSPVSDHGLVESCKYVEDQTENLRIYKVNEMGICLNFYAKCDMRTKATQCICPCAGQAWRWYPDMMCPFGHLIPLIMGCTWIAIGWDMASAFGWKMRHEECWGAMDCKMKQAEQTNEIIWWLEHQMHPAMKSYGFPKVSPHRCTQREECNFVNRWKNISQMHWYVHQNQQGPGMLWFHEYADRAFCGSGSMGQYDHDRPYMKWFMHNLQYWAHIIECQPGMSSGHIDDPKIGTIIVMAGIGGYDQDNIDNNDKESLVWIFRLQPWRYPFVRKLPPKIWNPVDAPLSWLQGPRIMSTCDRTELNGMTTCQMPDDFPDLESMYEMYERVIMIKCDYMDDCYERNEYCYMIPAELTEFWIMHIDAYMGLQAAIDSIEDLYHEKGDTHCMAEMRSAMYGDWFGLHCPFYGLEEWGLTQKKILVDHRDLDTFTHNGAHIYWSQMIVADHFWIENKMMYFIDCKDANCHIIFVTLWNRRCVGFQNNCWYCKWCKFCLPKVDVWKKPGWNIEYEIYDEVYCVAQVWFNGTTKKIAAMAMVRSDISDEKDWPRRQDCYLTGLTLQFDQYYPKLGFKFSAFYNNMLPMFPMWEKFNNHSHETWLDDHYNGNYYRQVMFLAFNCASPYRPATEKTNRHANGNCVIDHRTRWFIVMGRQKQRVASCGSMAGDVCVNMHHCTVPAWALQAFSTKVNNGHIHQWAIETHPVWNMVPHGVLPLECCRVRQNWHTDQYNKQEEVWKFPKTKPETRPPDEDPDSCWDRKEGERGQGADGMLRETNERHWCVIRYHCAARQERANCFFHFVKVQVCPCTVIYFMPAVEKTGTFRTSDYGIFWRWCCQWCEQMFKSGIMVGSGHAWEFHMFPDAPIFCDIGSGPDPDSWWHVYEWYGHESYTPINQNVKFDQEKHYNNAWAFCYIGAYTSTFLWVELGVYRCWWKTELERKLLDEFIVVCSNSYWEWPKADDSMHYMPDDNRWTVWELKYGRWAHRAQVTIDVFTCHYGFQYANSTLEIFDDPEYTKNHASQVMIDDLGFVLCDNNRHGSTSVIYSCAQDLPNGEDKMCPWAHASYNVGYDWLIHDVYLPKKTNPIMYMVQWLQMRWEMVYPMYSDTYWNPQSRGHFWVCQDWHSYGLYCNPSYPSIGCWYMTEAFTGCIWGSFHMLHWAHKQLCEIYEPQCWSHIVCCKSPCKLMPYWGYEIDQGTADINMEYQEKAYEQCFQVQSYKIKMHAQEYEASTFDRDITSPRKCAIIFPVGVRPRGVQLLCAQQHAMTEGMWLSERLLLTAWLRHRCSRKYGHKQRFETLCRSVNFISPHEVKEAWTWYYYNNGWMDKCWEQGSGGWAWYIGEMNAHHSEWLSINYQPTCTPVCYQVDTTTPHGMPWGLQHNHVYRVQIQTTNHEVGPFQNDPWKSWTAAFDGPNCQCCETGTEFGPCTFIRMKYGYVFWKSYYKFPWWLPSAILHGAFRSFFFKCGVWWVTCKTLTSLALKFINTQPKEYKAGIEIHAYPSNYELLRCFVFYLYNRRTWCFNIWWNWHVEFESKQYMWENDYSNSQQFHSTKMKREQFKETWVFKFPSAQEDDERKPLTSDTRMDLDGQTYVAQHLGVCSPSYGNIGDDHDDMLFTHHRRFHFGGWQIPENFKPDHGKEYVNFYIALSERAERYIPKHLINAWAQDRRWTWMITWMTAGYYWKTPFCPEPEDDNNKFPSHFPHTYKQKLFYRKVYKIDLLLKMHARLAECMCCGRAINMRKGDRADHMMAMFLEWWQ'
y='FGVLMCLSDLFQYMRNSAEAYTDTAKWKFYVMYQSDQWQRHGFFPPCTNPQNSHAWSCSSRYGFWWIKTDILRMLHEWRFPVCNRRSYQPCCSSYPICCPKWNVFNIKHTVETYKMLYCHDGYNCNHRLFHSSMVKRAVLMTQLLWLEWMLGCFRSSKYKHCIYRMIQNPPPNVMSYHIMPQQVFNPNIYYYGQYAADSAISHPILVPLQHQTEPFFTHHNWDRPHYMSTNDYKEIVYRCAEDNSKATNCWRYNTIWHDAMLLMTSYSAEKSHCFVNNTQPEVAAHEMQSNVDTSRDILSTQCKDASYPEVNNQGTGNVDQGEPYQFWEREKTWKVFHGYMIITQYNMPHRHWGWHVDFFEVWEVHSIFDAVRFEYDEIMRRSWATWPQNTGFTFSFTTTWNNQTDFLKMHDDNPWLSGKQWFPEHHHSHNMNNYDCEWMVMCDKKVDVIIDLREQTTQFTPECWLADAKYFKEFWGTTRRNWIAKCYIIDGRRMMQQVERGPHMQEVCKDICGKIPPWYHFMDSKVMLARQAGEPFRVVHPPMHYARIDYDCACWGMGWNRYACITQSLMCEMEKHMKVRPSDMTNMLRYEDMVRRTWTLLGREALTSICNDIWFLKHPWMVRHWFYAVFHNDLQSAQFWIPNATHKVDVEITRHFQLAWHRICHGHQCDNKQLNRIIKKWCRTMLTIFHKWAGFKWSFRWNVHYSYKDHGAEISHVRGEVWTNLHSLMGVTGYWLPDLGQKCCLIRINVDPYGQWTHGWCRCRIPWQYMKDTGFHRLRMGDMDMKSNCPAEWQHNIIHFVLRLRDRIGNMEVNDSCQVRWEAGKSHTGSIGPACKRNKGDRFMRESTAFNGSERSNEWEVNPMDLLGWNTGHFAMVENAHDVWSACTWDSMHHHLINGQMLLACHMVFWHEKQQFIWMSHSINLTHHSLDTSKFLAAVWYWVGWMTRVLQVNVPSLLHIDYWDAAQGFASVNCIMCVHPTVGHWYNMQWVNCDVGDCSMETGFGATVQDLMRECLWRATPGFECKIMKDYQYKLWNTNYYRYMRSQAEWDMWCDLAYLRFYDFIMECCTSRKLWCCCEVDFRKIVQDRQDEEDKEFSAWCMWTNKIWCHRVIVFKLCHGGVVFNCMSHMYYTVYIPVPAPWMPIYKYGIRTYVDKIFPDQGVWQLENMNGMPILYMMPQSHNWDMKLDHCWVKNTTNRIQMAAELQTTGSSNAGLSVMPFLQMRMLEFEQSSAQQRLMVAHNRDGPMMTIKEEIRVDEWAHSAACYQRNERKQPTQHMWWSRFKNTKNMFNCNKEKEFPYGYYSEHCPRFDDSRMNWVFDYHASCMDCNYRCVIMQRWGHGHYARRHCQSFHLTSFDDIIVKQMYWWAYVNCTQRRCYVFKLKYSFAPKKYYIPVDCYQCWYPNHSVMWRTDYLENASKSCHQFWWQTQNCHLKLSFVRLHKYFHFQPFGEPPSPIGCATRIANNQYVYESKSVWFFNQKFWECQPCWWLGKPYWFQMATTHTKHIDYITTSGRSCDLMAVQSKQRLPHFGSNELQVKSMCRIQCIWPIEPYKWEAQVQGNHFIVQKGMWIWTFKGMNRCESQEDCGTYWGCGNWQNCPPFALSKYEKNEAHWSYSRLCCVRPMFFYMVRDSEGRMHSYAIVATFRRQKFFNCFSIVHGFRQQCDLWMMDAFDSYWAAFVSSVDCIGKTDNHERTEADRFAMFAKNDWKRREDRYWRYATMVGLKAWPTVRTENGHVCHQNTKKIKPWNHIHAFFWECTWTWVRCINAVYDTMPNRYNNLMPIEENVNLMPITLDDPRYQTAELYFQGQQHFDLLCKHYYHYLNPQCVQHKVQENTGYEWPQATAYHYVDTDQKKGCIITEAMQHIKQENVLKPLVWPVRGRIRISVCNGWHYKQEKCFVMAYGVCKNLQLLSHEEPMNSGHIWTWNRSGCYFLKWFPFKAKHLFATNKYRQDNFVKRAGFNCCGHYCVGMINCAPNLTLNAVQMCQQAEYHYCDIPKFTSLMWLVDEGTVKTAQACWNQIMSDPFCGRRIGVSTVGQGYIHDLAMNPKTYQDVINNDEQWSSHYFIILWRWVGVTSLSFHFDTDRFSTNFWNSLRISKMLNPMCTPPYLTNVWEPSCYSQNAYYHQFEAKEQLCIGAFKQYFTPHCQAADWRWRCDNDSPVSDHGLVDKVNFMGISLNFYAKCDMCLCAGQAWRWYPDMMCPFTYICTWIAIGWDMWSAFGWKMRHPWGRGYQTQGCKMKQAEYLNEILEHQHQPAMKSYGEDLACKLSNSPHRFTQREECNFVNRWKNISQMHQYDGMLVFHEYADCFFAARAFCGSGDKRVKMVMGQYHHDRPYMPCWMWRWIEFDVHQPWMSSGHIGCNCDEKIGTIIVYDQDNIDNNDKESLVWITRCAHVCGPWRNHHCGPPKKIWPPVDATLSWLILRWGPRICSTCGMCQMPDDFPDLMYERVIDYMDDCYERYEFYYYKAMEMYMQPAELGEFWIMHIDAYMGEQWAIDSIEDLYHEKGDTHCMSEMSAMYGDWFGQDPLRFHRCPFFQNSGGYKYFFYTCEEWTQKKILVDHRTGLIKNIFNFGQIMVDTFYHNGASHFWIENKMMYFIDCKDANCHICVHWVQMVRNVTLWNVNRCVGFRNNCWYHLVAWIKWCKFWLPKWKKPGWNPEYEINDEDTGTSIYCEWWYVAQVNGTTKKCIAMVFVYANEGMVRSDIDCYLTCHTLILTLQFDQYYPKLGFKFSAFYNNMLPMFPMWEKFNNHSHSTWLADHYNGPLFDAGGYTRQVMFTDTMRAWFNCASPYRPATEKTERHANGRCVIRTRWHIVMGRQKQRVHRCSEWVTKYTGSMAGFVCVVPAWAQPTNSNGIHRCGHEPFKPVWNMVEWTYRSNTQKNVLECCRVRQNWHTDVWETRKPDEDIKGMHNIDQSCWDRCEGERGHTAQFQYIGAVGMLRETNERKWCVSPLAIFICERYHCPIQGSVEHARQERANCGFHFVKVKVCPCTVIYFMPAVEKTGTFRPRRAQCSDYGRCRWRWCCQWCKLMFKSNQWVFRGIPVGSGHAWEPHMFPDAPIFPLSWFLTEWWGSCRYYHWVSYTDMCWYKLYLCVTNNAWSWNYIGAYTSTFLWVELGVYRCWWKTDEFIVVCSNSYWEHPKADDSMHYMMKCAYDDHRWTVWELKYGRIEKYRRDDKNIDVTKTCHYGFQYANSTLGADEIFDDHEYTKNHALQMMIDDGGFTLSCAQDLPNGEDKMCPWAHASMEVPSIVSDWLIHDVYLPYFMQWLMQSHKQMIWEMVYPMYSDTYWNPQSRGHFWWCQDWHSYGVCGLYCNPSYPSINIETCQRTNTQYMTEAFTGCIWGAHKQLCEIYDPQCWSHINITIPPTCKSPCKLMRWNNDGIKNMCPWLIWLGGKGTADINMMYKAYWCRQVQSYKLKMQAQEYCASTFDRAITSPRPGAIIFPVGLCAMTEGMWLSEVLLLTGWLRHRCIRIYGHKQRFETLCRSVNFISPMEVVPVVVIFGLAWTWYYYNNLWMNSPQWYWETHMYWTANGSRGWSWYIGEMNWLSINYQPTCTFPYHSFVYQVATTTPHGMPWGLQHNIVYRVQIQTSNHPVGPFQNDPWWTAAFDGPNCQCCETGTEFGPCTAFYQSIRMKYGGFACLWTIHVFPLPSAILHGFFFKCKIVEHWTPMNWWDKTLTSSACKFINRSWHGRDQPKEYKAGIETHVKWSNYELLCCFVFYYYNRKTWCFNIWWNWHVEFESKYYEKYFYMGEENGWVMDNSNSQGFHQQETWVFKFPSAQEDDERKPLTSDTRMDLDDSGMAQTREMAVAQNLIGDIHDDVLFTVHRRFHFGGWQIPENNRQLQWQCAETAEIPKTAQDRRRTIAMCKQMIPWMNAGYYWKTPFCPECEWNDNCCSSHFPKPGWSHGWWGLLAECMCCGRAINMTKGDHAMAMFLEWWQ'
print(len(x),len(y))
### Don't run, too slow
#Fmat = scoringMatrix(x,y,indel=-5)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.91 µs
3924 3943


In [20]:
def scoringMatrix(x,y,indel=-5):
    m = len(x)
    n = len(y)
    Fmatrix = np.zeros((m+1,n+1))

    ## initialize
    for i in range(1,m+1):
        Fmatrix[i,0] += Fmatrix[i-1,0] + indel
    for j in range(1,n+1):
        Fmatrix[0,j] += Fmatrix[0,j-1] + indel

    ##
    for i in range(1,m+1):
        for j in range(1,n+1):
            if x[i-1] == y[j-1]:
                Fmatrix[i,j] = Fmatrix[i-1,j-1] + scorematch(x[i-1],y[j-1],blosum)
            else:
                mm = Fmatrix[i-1,j-1] + scorematch(x[i-1],y[j-1],blosum)
                down = Fmatrix[i-1,j] + indel
                right = Fmatrix[i,j-1] + indel
                Fmatrix[i,j] = max([mm,down,right])
            #print(x[i-1],y[j-1],[mm,down,right])


    #pd.DataFrame(Fmatrix,columns=['-']+[_ for _ in y],index=['-']+[_ for _ in x])
    ##
    i = Fmatrix.shape[0] - 1
    j = Fmatrix.shape[1] - 1

    LCS = ""
    a = ''
    b = ''
    paths = []
    while (j>0 or i>0):
        if x[i-1] == y[j-1]:
            #print(x[i-1],y[j-1],Fmatrix[i,j])
            LCS += x[i-1]
            j -=1
            i -=1
            a +=x[i]
            b +=y[j]

            paths.append([(np.inf),(x[i],y[j]) ])
        else:
            diag = Fmatrix[i-1,j-1] + scorematch(x[i-1],y[j-1],blosum)
            left = Fmatrix[i-1,j] + indel
            up = Fmatrix[i,j-1] + indel
            maxCell = np.argmax([diag,left,up])
            #print(maxCell,'diff',[diag,left,up])
            if maxCell == 0:
                j -=1
                i -=1
                a +=x[i]
                b +=y[j]
            else:
                if maxCell == 1:
                    i -= 1
                    a += x[i]
                    b += '-'
                else:
                    j -= 1
                    a += '-'
                    b += y[j]
            paths.append( [(diag,left,up),
                           (x[i],x[i],"-"),
                           (y[j],"-",y[j]) ])


    #print('Score',Fmatrix[m][n])        
    #print(LCS[::-1]) #AACTGG
    #print('--')
    #print(b[::-1])
    #print(a[::-1])
    score = Fmatrix[m][n]
    LCS = LCS[::-1]
    a = a[::-1]
    b = b[::-1]
    return score, LCS, a,b,Fmatrix

In [21]:
%time
x='PLEASANTLY'
y='MEANLY'
print(len(x),len(y))
scoringMatrix(x,y,indel=-5)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
10 6


(8.0,
 'EANLY',
 'PLEASANTLY',
 '-ME--AN-LY',
 array([[  0.,  -5., -10., -15., -20., -25., -30.],
        [ -5.,  -2.,  -6., -11., -16., -21., -26.],
        [-10.,  -3.,  -5.,  -7., -12., -12., -17.],
        [-15.,  -8.,   2.,  -3.,  -7., -12., -14.],
        [-20., -13.,  -3.,   6.,   1.,  -4.,  -9.],
        [-25., -18.,  -8.,   1.,   7.,   2.,  -3.],
        [-30., -23., -13.,  -4.,   2.,   6.,   1.],
        [-35., -28., -18.,  -9.,   2.,   1.,   4.],
        [-40., -33., -23., -14.,  -3.,   1.,  -1.],
        [-45., -38., -28., -19.,  -8.,   1.,   0.],
        [-50., -43., -33., -24., -13.,  -4.,   8.]]))

In [144]:
# x='ACGGAA'
# y='ATTCAA'
x="CCAATCTACTACTGCTTGCA"
y="GCCACTCTCGCTGTACTGTG"
indel = 0
m = len(x)
n = len(y)
Fmatrix = np.zeros((m+1,n+1))

# ## initialize
for i in range(1,m+1):
    Fmatrix[i,0] += Fmatrix[i-1,0] + indel
for j in range(1,n+1):
    Fmatrix[0,j] += Fmatrix[0,j-1] + indel

for i in range(1,m+1):
    for j in range(1,n+1):
        if x[i-1] == y[j-1]:
            Fmatrix[i,j] = Fmatrix[i-1,j-1] + 1
        else:
            mm = Fmatrix[i-1,j-1] + 0
            down = Fmatrix[i-1,j] + indel
            right = Fmatrix[i,j-1] + indel
            Fmatrix[i,j] = max([mm,down,right])
Fmatrix

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 0.,  0.,  1.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
         2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
       [ 0.,  0.,  1.,  2.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
         3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
       [ 0.,  0.,  1.,  2.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
         3.,  3.,  4.,  4.,  4.,  4.,  4.,  4.],
       [ 0.,  0.,  1.,  2.,  3.,  3.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
         4.,  4.,  4.,  4.,  5.,  5.,  5.,  5.],
       [ 0.,  0.,  1.,  2.,  3.,  4.,  4.,  5.,  5.,  5.,  5.,  5.,  5.,
         5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.],
       [ 0.,  0.,  1.,  2.,  3.,  4.,  5.,  5.,  6.,  6.,  6.,  6.,  6.,
         6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.],
       [ 0.,  0.,  1.,  

In [60]:
x='ACGGAA'
y='ATTCAA'
m=len(x)
n=len(y)
initM = pd.DataFrame(0,index=range(0,m+1),columns=range(0,n+1))
linMat = np.zeros((m+1,n+1))

In [47]:
print(initM)

   0  1  2  3  4  5  6
0  0  0  0  0  0  0  0
1  0  0  0  0  0  0  0
2  0  0  0  0  0  0  0
3  0  0  0  0  0  0  0
4  0  0  0  0  0  0  0
5  0  0  0  0  0  0  0
6  0  0  0  0  0  0  0


In [61]:
print(linMat)

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


In [64]:
for i in range(1,len(y)+1):
    linMat[:,i-1]=i
    
linMat

array([[1., 2., 3., 4., 5., 6., 0.],
       [1., 2., 3., 4., 5., 6., 0.],
       [1., 2., 3., 4., 5., 6., 0.],
       [1., 2., 3., 4., 5., 6., 0.],
       [1., 2., 3., 4., 5., 6., 0.],
       [1., 2., 3., 4., 5., 6., 0.],
       [1., 2., 3., 4., 5., 6., 0.]])

In [63]:
Fmatrix

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 2., 2., 2.],
       [0., 1., 1., 1., 2., 2., 2.],
       [0., 1., 1., 1., 2., 2., 2.],
       [0., 1., 1., 1., 2., 3., 3.],
       [0., 1., 1., 1., 2., 3., 4.]])

In [76]:
Fmatrix[:,0]

array([0., 0., 0., 0., 0., 0., 0.])

In [101]:
a="-"+x
b="-"+y
m = len(x)
n = len(y)
indel = -5

In [102]:
## initialize
byCol = np.zeros((1,m+1))
byRow = np.zeros((n+1,1))

for i in range(1,m+1):
    byCol[0,i] += byCol[0,i-1] + indel

for j in range(1,n+1):
    byRow[j,0] += byRow[j-1,0] + indel

print(byCol)
print(byRow)

[[  0.  -5. -10. -15. -20. -25. -30.]]
[[  0.]
 [ -5.]
 [-10.]
 [-15.]
 [-20.]
 [-25.]
 [-30.]]


In [132]:
def score(i,j):
    if x[i] == y[j]:
        #print(a[i],b[j])
        return 1
    else:
        up = byCol[0,i-1]
        diag = byRow[i-1,0]
        left = byRow[i,0]
        return max(up,diag,left)

In [142]:
xPos=0
a=[score(xPos,i) for i,j in enumerate(y)]


TypeError: 'CClass' object is not callable

In [66]:
for i in range(1,len(y)+1):
    print(linMat[:,i-1:i+1])
    



[[1. 2.]
 [1. 2.]
 [1. 2.]
 [1. 2.]
 [1. 2.]
 [1. 2.]
 [1. 2.]]
[[2. 3.]
 [2. 3.]
 [2. 3.]
 [2. 3.]
 [2. 3.]
 [2. 3.]
 [2. 3.]]
[[3. 4.]
 [3. 4.]
 [3. 4.]
 [3. 4.]
 [3. 4.]
 [3. 4.]
 [3. 4.]]
[[4. 5.]
 [4. 5.]
 [4. 5.]
 [4. 5.]
 [4. 5.]
 [4. 5.]
 [4. 5.]]
[[5. 6.]
 [5. 6.]
 [5. 6.]
 [5. 6.]
 [5. 6.]
 [5. 6.]
 [5. 6.]]
[[6. 0.]
 [6. 0.]
 [6. 0.]
 [6. 0.]
 [6. 0.]
 [6. 0.]
 [6. 0.]]


In [349]:
x='A'
y='ATTCAA'
m = len(x)
n = len(y)
indel = 0

## Initialize
top = [0]
score = np.cumsum([indel] + [indel for _ in y])
score

array([0, 0, 0, 0, 0, 0, 0])

In [265]:
# x = 'A'
# match = top + [ 1 if _==x else 0 for i,_ in enumerate(y)]
# print(match)

[0, 1, 0, 0, 0, 1, 1]


In [266]:
# for i in range(1,n+1):
#     #print(score[i],match[i],score[i-1],max(score[i],match[i],score[i-1]))
#     score[i] = max(score[i],match[i],score[i-1])
#     #left[i] = max(match[i],left[i],left[i-1]) 
# score
     

array([0, 1, 1, 1, 1, 1, 1])

In [284]:
x = 'A'
match = top + [ 1 if _==x else 0 for i,_ in enumerate(y)]
print(match)

[0, 1, 0, 0, 0, 1, 1]


In [312]:
score = np.cumsum([indel] + [indel for _ in y])
def linSpace(x,score,match=1,indel=0):
    for i in range(1,n+1):
        #print(i,score,match[i], score[i], score[i-1])
        if x == y[i-1]:
            score[i] += match
        else:
            score[i] = max(score[i] + indel,score[i-1]+indel)
    return(score)

In [347]:
score = np.cumsum([indel] + [indel for _ in y])

In [348]:
x='A'
score = linSpace(x,score)
score

TypeError: linSpace() missing 1 required positional argument: 'score'

In [308]:
x='C'
score = linSpace(x,score)
score

array([0, 1, 1, 1, 2, 2, 2])

In [309]:
x='G'
score = linSpace(x,score)
score

array([0, 1, 1, 1, 2, 2, 2])

In [310]:
x='G'
score = linSpace(x,score)
score

array([0, 1, 1, 1, 2, 2, 2])

In [311]:
x='A'
score = linSpace(x,score)
score

array([0, 2, 2, 2, 2, 3, 3])

In [566]:
x='A'
score = linSpace(x,score)
score

TypeError: linSpace() missing 1 required positional argument: 'score'

In [292]:
for i in range(1,n+1):
    print(i,score,match[i], score[i], score[i-1])
    if x == y[i-1]:
        score[i] += 1
    else:
        score[i] = max(score[i-1], score[i])
score

1 [0 0 0 0 0 0 0] 1 0 0
2 [0 1 0 0 0 0 0] 0 0 1
3 [0 1 1 0 0 0 0] 0 0 1
4 [0 1 1 1 0 0 0] 0 0 1
5 [0 1 1 1 1 0 0] 1 0 1
6 [0 1 1 1 1 1 0] 1 0 1


array([0, 1, 1, 1, 1, 1, 1])

In [263]:
for i in range(1,n+1):
    #print(score[i],match[i],score[i-1],max(score[i],match[i],score[i-1]))
    whichMax = np.argmax([score[i-1],match[i],score[i],])
    print(score[i],match[i],score[i-1])
    if match[i] == 1 and whichMax == 0: 
        print(i)
        score[i] += score[i-1]
    else: score[i] += indel    
    #left[i] = max(match[i],left[i],left[i-1]) 
score
     

1 0 0
1 0 1
1 0 1
2 1 1
1 0 2
1 0 1


array([0, 1, 1, 1, 2, 1, 1])

In [350]:
x='ACGGAA'
y='ATTCAA'
m = len(x)
n = len(y)
Fmatrix = np.zeros((m+1,n+1))

## initialize
for i in range(1,m+1):
    Fmatrix[i,0] += Fmatrix[i-1,0] + 0
for j in range(1,n+1):
    Fmatrix[0,j] += Fmatrix[0,j-1] + 0

##
for i in range(1,m+1):
    for j in range(1,n+1):
        if x[i-1] == y[j-1]:
            Fmatrix[i,j] = Fmatrix[i-1,j-1] + 1
        else:
            mm = Fmatrix[i-1,j-1] + 0
            down = Fmatrix[i-1,j] + 0
            right = Fmatrix[i,j-1] + 0
            Fmatrix[i,j] = max([mm,down,right])
            
Fmatrix

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 2., 2., 2.],
       [0., 1., 1., 1., 2., 2., 2.],
       [0., 1., 1., 1., 2., 2., 2.],
       [0., 1., 1., 1., 2., 3., 3.],
       [0., 1., 1., 1., 2., 3., 4.]])

In [600]:

def linSpaceAlignOcFiv(residue,y,updatedScore,nullScore,match=1,indel=0,mismatch=0,j=0,verbose=False):    
    
    n = len(y)

    updatedScore = [nullScore[j]] + updatedScore
    prevScore = updatedScore.copy()

    #(prevScore,updatedScore)

    for i in range(1,n+1):
        if residue == y[i-1]:
            if verbose: print(i, 'match',y[i-1], prevScore,updatedScore)
            updatedScore[i] = prevScore[i-1] + match
        else:
            if verbose: print(i,'no match', y[i-1], prevScore,updatedScore,'choose',
                                  updatedScore[i-1],
                                  updatedScore[i],prevScore[i-1])
            updatedScore[i] = max(updatedScore[i-1] + indel, #Up
                                  updatedScore[i] + indel, #Left
                                  prevScore[i-1] + mismatch) #Mismatch
        if verbose: print(updatedScore,"\n----")
    return updatedScore[1:]

In [627]:
score = list(np.cumsum([indel for _ in y]))
nullScore = list(np.cumsum([0] + [indel for _ in x]))


A0=linSpaceAlignOcFiv('A',y,score,nullScore,verbose=False)
print(A0)
C1=linSpaceAlignOcFiv('C',y,A0,nullScore,verbose=False)
print(C1)
G34 = linSpaceAlignOcFiv('G',y,C1,nullScore,verbose=False)
print(G34)
print(G34)
A5 = linSpaceAlignOcFiv('A',y,G34,nullScore,verbose=False)
print(A5)

[1, 1, 1, 1, 1, 1]
[1, 1, 1, 2, 2, 2]
[1, 1, 1, 2, 2, 2]
[1, 1, 1, 2, 2, 2]
[1, 1, 1, 2, 3, 3]


In [628]:
score = list(np.cumsum([indel for _ in y]))
nullScore = list(np.cumsum([0] + [indel for _ in x]))

dScores = {0:score}
for i in range(1,len(x)+1):
    dScores[i] = linSpaceAlignOcFiv(x[i-1],y,dScores[i-1],nullScore,verbose=False,j=i-1)

pd.DataFrame.from_dict(dScores.values()).T

,0,1,2,3,4,5,6
0,0,1,1,1,1,1,1
1,0,1,1,1,1,1,1
2,0,1,1,1,1,1,1
3,0,1,2,2,2,2,2
4,0,1,2,2,2,3,3
5,0,1,2,2,2,3,4


In [599]:
updatedScore = list(np.cumsum([0] + [indel for _ in y]))
nullScore = list(np.cumsum([0] + [indel for _ in x]))
linSpaceAlignOcFiv('A',y,A0,nullScore)

IndexError: list assignment index out of range

In [570]:
x='ACGGAA'
y='ATTCAA'
indel = 0
score = list(np.cumsum([indel for _ in y]))
n = len(y)
### top row on the initialization F-matrix. Cumulative sum of indels 
nullScore = list(np.cumsum([0] + [indel for _ in x]))
(nullScore,score)

([0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0])

In [571]:
updatedScore = [nullScore[0]] + score
prevScore = updatedScore.copy()

(prevScore,updatedScore)

([0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0])

In [572]:
residue = 'A'
for i in range(1,n+1):
    if residue == y[i-1]:
        print(i, 'match',y[i-1], prevScore,updatedScore)
        updatedScore[i] = prevScore[i-1] + 1
    else:
        print(i,'no match', y[i-1], prevScore,updatedScore,'choose',
                              updatedScore[i-1],
                              updatedScore[i],prevScore[i-1])

        updatedScore[i] = max(updatedScore[i-1] + indel, #Up
                              updatedScore[i] + indel, #Left
                              prevScore[i-1]) #Mismatch
    print(updatedScore,"\n----")
updatedScore

1 match A [0, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0] 
----
2 no match T [0, 0, 0, 0, 0, 0, 0] [0, 1, 0, 0, 0, 0, 0] choose 1 0 0
[0, 1, 1, 0, 0, 0, 0] 
----
3 no match T [0, 0, 0, 0, 0, 0, 0] [0, 1, 1, 0, 0, 0, 0] choose 1 0 0
[0, 1, 1, 1, 0, 0, 0] 
----
4 no match C [0, 0, 0, 0, 0, 0, 0] [0, 1, 1, 1, 0, 0, 0] choose 1 0 0
[0, 1, 1, 1, 1, 0, 0] 
----
5 match A [0, 0, 0, 0, 0, 0, 0] [0, 1, 1, 1, 1, 0, 0]
[0, 1, 1, 1, 1, 1, 0] 
----
6 match A [0, 0, 0, 0, 0, 0, 0] [0, 1, 1, 1, 1, 1, 0]
[0, 1, 1, 1, 1, 1, 1] 
----


[0, 1, 1, 1, 1, 1, 1]

In [579]:
updatedScore = [nullScore[0]] + updatedScore[1:]
prevScore = updatedScore.copy()

(prevScore,updatedScore)

residue = 'A'
for i in range(1,n+1):
    if residue == y[i-1]:
        print(i, 'match',y[i-1], prevScore,updatedScore)
        updatedScore[i] = prevScore[i-1] + 1
    else:
        print(i,'no match', y[i-1], prevScore,updatedScore,'choose',
                              updatedScore[i-1],
                              updatedScore[i],prevScore[i-1])

        updatedScore[i] = max(updatedScore[i-1] + indel, #Up
                              updatedScore[i] + indel, #Left
                              prevScore[i-1]) #Mismatch
    print(updatedScore,"\n----")
updatedScore

1 match A [0, 1, 1, 1, 2, 3, 3] [0, 1, 1, 1, 2, 3, 3]
[0, 1, 1, 1, 2, 3, 3] 
----
2 no match T [0, 1, 1, 1, 2, 3, 3] [0, 1, 1, 1, 2, 3, 3] choose 1 1 1
[0, 1, 1, 1, 2, 3, 3] 
----
3 no match T [0, 1, 1, 1, 2, 3, 3] [0, 1, 1, 1, 2, 3, 3] choose 1 1 1
[0, 1, 1, 1, 2, 3, 3] 
----
4 no match C [0, 1, 1, 1, 2, 3, 3] [0, 1, 1, 1, 2, 3, 3] choose 1 2 1
[0, 1, 1, 1, 2, 3, 3] 
----
5 match A [0, 1, 1, 1, 2, 3, 3] [0, 1, 1, 1, 2, 3, 3]
[0, 1, 1, 1, 2, 3, 3] 
----
6 match A [0, 1, 1, 1, 2, 3, 3] [0, 1, 1, 1, 2, 3, 3]
[0, 1, 1, 1, 2, 3, 4] 
----


[0, 1, 1, 1, 2, 3, 4]

In [517]:
updatedScore = [nullScore[1]] + updatedScore[1:]
prevScore = updatedScore.copy()

print(prevScore)
print(updatedScore)

[-1, 1, 0, -1, -2, -3, -4]
[-1, 1, 0, -1, -2, -3, -4]


In [565]:
x='ACGGAA'
y='ATTCAA'
indel = 0
score = list(np.cumsum([indel for _ in y]))
n = len(y)

### top row on the initialization F-matrix. Cumulative sum of indels starting at 0, with length m + 1. 
nullScore = list(np.cumsum([0] + [indel for _ in x]))
(nullScore,score)


#### Initialize
updatedScore = [nullScore[0]] + score
prevScore = updatedScore.copy()

for j,residue in enumerate(x):
    updatedScore = [nullScore[j]] + updatedScore[1:]
    prevScore = updatedScore.copy()
    print(residue)
    for i in range(1,n+1):
        if residue == y[i-1]:
            print(i, 'match',y[i-1], prevScore,updatedScore)
            updatedScore[i] = prevScore[i-1] + 1
        else:
            print(i,'no match', y[i-1], prevScore,updatedScore,'choose',
                                  updatedScore[i-1],
                                  updatedScore[i],prevScore[i-1])

            updatedScore[i] = max(updatedScore[i-1] + indel, #Up,
                                  updatedScore[i] + indel, #Left,
                                  prevScore[i-1]) #Mismatch
        print(updatedScore,"\n----")

finalScore = [nullScore[j+1]] + updatedScore[1:]
print('final',finalScore)

A
1 match A [0, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0] 
----
2 no match T [0, 0, 0, 0, 0, 0, 0] [0, 1, 0, 0, 0, 0, 0] choose 1 0 0
[0, 1, 1, 0, 0, 0, 0] 
----
3 no match T [0, 0, 0, 0, 0, 0, 0] [0, 1, 1, 0, 0, 0, 0] choose 1 0 0
[0, 1, 1, 1, 0, 0, 0] 
----
4 no match C [0, 0, 0, 0, 0, 0, 0] [0, 1, 1, 1, 0, 0, 0] choose 1 0 0
[0, 1, 1, 1, 1, 0, 0] 
----
5 match A [0, 0, 0, 0, 0, 0, 0] [0, 1, 1, 1, 1, 0, 0]
[0, 1, 1, 1, 1, 1, 0] 
----
6 match A [0, 0, 0, 0, 0, 0, 0] [0, 1, 1, 1, 1, 1, 0]
[0, 1, 1, 1, 1, 1, 1] 
----
C
1 no match A [0, 1, 1, 1, 1, 1, 1] [0, 1, 1, 1, 1, 1, 1] choose 0 1 0
[0, 1, 1, 1, 1, 1, 1] 
----
2 no match T [0, 1, 1, 1, 1, 1, 1] [0, 1, 1, 1, 1, 1, 1] choose 1 1 1
[0, 1, 1, 1, 1, 1, 1] 
----
3 no match T [0, 1, 1, 1, 1, 1, 1] [0, 1, 1, 1, 1, 1, 1] choose 1 1 1
[0, 1, 1, 1, 1, 1, 1] 
----
4 match C [0, 1, 1, 1, 1, 1, 1] [0, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 2, 1, 1] 
----
5 no match A [0, 1, 1, 1, 1, 1, 1] [0, 1, 1, 1, 2, 1, 1] choose 2 1 1
[0, 1, 1, 

In [529]:
indel =0 
n = len(y)
nullScore = list(np.cumsum([0] + [indel for _ in x]))
score = list(np.cumsum([indel for _ in y]))


updatedScore = [nullScore[0]] + score
prevScore = updatedScore.copy()
    
residue = 'A'
for i in range(1,n+1):
    if residue == y[i-1]:
        print(i, 'match',y[i-1], prevScore,updatedScore)
        updatedScore[i] = prevScore[i-1] + 1
    else:
        print(i,'no match', y[i-1], prevScore,updatedScore,'choose',
                              updatedScore[i-1],
                              updatedScore[i],prevScore[i-1])

        updatedScore[i] = max(updatedScore[i-1] + indel, #Up,
                              updatedScore[i] + indel, #Left,
                              prevScore[i-1]) #Mismatch
        
updatedScore = [nullScore[j]] + updatedScore[1:]
prevScore = updatedScore.copy()
(updatedScore,prevScore)

1 match A [0, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0]
2 no match T [0, 0, 0, 0, 0, 0, 0] [0, 1, 0, 0, 0, 0, 0] choose 1 0 0
3 no match T [0, 0, 0, 0, 0, 0, 0] [0, 1, 1, 0, 0, 0, 0] choose 1 0 0
4 no match C [0, 0, 0, 0, 0, 0, 0] [0, 1, 1, 1, 0, 0, 0] choose 1 0 0
5 match A [0, 0, 0, 0, 0, 0, 0] [0, 1, 1, 1, 1, 0, 0]
6 match A [0, 0, 0, 0, 0, 0, 0] [0, 1, 1, 1, 1, 1, 0]


([0, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1])

In [563]:
residue = 'A'

nullScore = list(np.cumsum([0] + [indel for _ in x]))
score = list(np.cumsum([indel for _ in y]))

A0=linspaceAlign('A',y,match=1,indel=0,mismatch=0,updatedScore=score,nullScore=nullScore,resPos=0)
C1=linspaceAlign('C',y,match=1,indel=0,mismatch=0,updatedScore=A0,nullScore=nullScore,resPos=1)
G2=linspaceAlign('G',y,match=1,indel=0,mismatch=0,updatedScore=C1,nullScore=nullScore,resPos=2)
G3=linspaceAlign('G',y,match=1,indel=0,mismatch=0,updatedScore=G2,nullScore=nullScore,resPos=3)
A4=linspaceAlign('A',y,match=1,indel=0,mismatch=0,updatedScore=G3,nullScore=nullScore,resPos=4)
A5=linspaceAlign('A',y,match=1,indel=0,mismatch=0,updatedScore=A4,nullScore=nullScore,resPos=5)

(A0,C1,G2,G3,A4,A5)

1 match A [0, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0]
2 no match T [0, 0, 0, 0, 0, 0, 0] [0, 1, 0, 0, 0, 0, 0] choose 1 0 0
3 no match T [0, 0, 0, 0, 0, 0, 0] [0, 1, 1, 0, 0, 0, 0] choose 1 0 0
4 no match C [0, 0, 0, 0, 0, 0, 0] [0, 1, 1, 1, 0, 0, 0] choose 1 0 0
5 match A [0, 0, 0, 0, 0, 0, 0] [0, 1, 1, 1, 1, 0, 0]
6 match A [0, 0, 0, 0, 0, 0, 0] [0, 1, 1, 1, 1, 1, 0]
1 no match A [0, 1, 1, 1, 1, 1, 1] [0, 1, 1, 1, 1, 1, 1] choose 0 1 0
2 no match T [0, 1, 1, 1, 1, 1, 1] [0, 1, 1, 1, 1, 1, 1] choose 1 1 1
3 no match T [0, 1, 1, 1, 1, 1, 1] [0, 1, 1, 1, 1, 1, 1] choose 1 1 1
4 match C [0, 1, 1, 1, 1, 1, 1] [0, 1, 1, 1, 1, 1, 1]
5 no match A [0, 1, 1, 1, 1, 1, 1] [0, 1, 1, 1, 2, 1, 1] choose 2 1 1
6 no match A [0, 1, 1, 1, 1, 1, 1] [0, 1, 1, 1, 2, 2, 1] choose 2 1 1
1 no match A [0, 1, 1, 1, 2, 2, 2] [0, 1, 1, 1, 2, 2, 2] choose 0 1 0
2 no match T [0, 1, 1, 1, 2, 2, 2] [0, 1, 1, 1, 2, 2, 2] choose 1 1 1
3 no match T [0, 1, 1, 1, 2, 2, 2] [0, 1, 1, 1, 2, 2, 2] choose 1 1 1
4 no match C [

([1, 1, 1, 1, 1, 1],
 [1, 1, 1, 2, 2, 2],
 [1, 1, 1, 2, 2, 2],
 [1, 1, 1, 2, 2, 2],
 [1, 1, 1, 2, 3, 3],
 [1, 1, 1, 2, 3, 4])

In [561]:
residue='C'
linspaceAlign(residue,y,match=1,indel=0,mismatch=0,updatedScore=A1,nullScore=nullScore,resPos=1)

1 no match A [0, 1, 1, 1, 1, 1, 1] [0, 1, 1, 1, 1, 1, 1] choose 0 1 0
2 no match T [0, 1, 1, 1, 1, 1, 1] [0, 1, 1, 1, 1, 1, 1] choose 1 1 1
3 no match T [0, 1, 1, 1, 1, 1, 1] [0, 1, 1, 1, 1, 1, 1] choose 1 1 1
4 match C [0, 1, 1, 1, 1, 1, 1] [0, 1, 1, 1, 1, 1, 1]
5 no match A [0, 1, 1, 1, 1, 1, 1] [0, 1, 1, 1, 2, 1, 1] choose 2 1 1
6 no match A [0, 1, 1, 1, 1, 1, 1] [0, 1, 1, 1, 2, 2, 1] choose 2 1 1


[1, 1, 1, 2, 2, 2]

In [567]:
def linspaceAlign(residue,y,match=1,indel=0,mismatch=0,updatedScore=None,nullScore=None,resPos=0):
    n = len(y)
    
    ### Define nullscore
    if nullScore is None:
        nullScore = list(np.cumsum([0] + [indel for _ in x]))
    
    
    ### Initialize vectors to keep track of scores
    if updatedScore is None:
        updatedScore = list(np.cumsum([indel for _ in y]))
    
    updatedScore = [nullScore[resPos]] + updatedScore
    
    ###
    
    
    prevScore = updatedScore.copy()

    
    ###
    for i in range(1,n+1):
        if residue == y[i-1]:
            print(i, 'match',y[i-1], prevScore,updatedScore)
            updatedScore[i] = prevScore[i-1] + match
        else:
            print(i,'no match', y[i-1], prevScore,updatedScore,'choose',
                                  updatedScore[i-1],
                                  updatedScore[i],prevScore[i-1])

            updatedScore[i] = max(updatedScore[i-1] + indel, #Up,
                                  updatedScore[i] + indel, #Left,
                                  prevScore[i-1] + mismatch) #Mismatch
    
    #updatedScore = [nullScore[resPos]] + updatedScore[1:]
    return updatedScore[1:]

In [523]:
x='ACGGAA'
y='ATTCAA'



### top row on the initialization F-matrix. Cumulative sum of indels starting at 0, with length m + 1. 


def linSpaceAlign(x,y,indel = -1,score=None){
    
    n = len(y)
    nullScore = list(np.cumsum([0] + [indel for _ in x]))
    if score is None:
        score = list(np.cumsum([indel for _ in y]))
    }


    #### Initialize
    updatedScore = [nullScore[0]] + score
    prevScore = updatedScore.copy()

for j,residue in enumerate(x):
    updatedScore = [nullScore[j]] + updatedScore[1:]
    prevScore = updatedScore.copy()
    
    for i in range(1,n+1):
        if residue == y[i-1]:
            print(i, 'match',y[i-1], prevScore,updatedScore)
            updatedScore[i] = prevScore[i-1] + 1
        else:
            print(i,'no match', y[i-1], prevScore,updatedScore,'choose',
                                  updatedScore[i-1],
                                  updatedScore[i],prevScore[i-1])

            updatedScore[i] = max(updatedScore[i-1] + indel, #Up,
                                  updatedScore[i] + indel, #Left,
                                  prevScore[i-1]) #Mismatch
        print(updatedScore,"\n----")

finalScore = [nullScore[j+1]] + updatedScore[1:]
print('final',finalScore)

[-6, -4, -3, -2, -1, 1, 3]

5

In [ ]:
residue = 'C'
for i in range(1,n+1):
    if residue == y[i-1]:
        print(i, 'match',y[i-1], prevScore,updatedScore)
        updatedScore[i] = prevScore[i-1] + 1
    else:
        print(i,'no match', y[i-1], prevScore,updatedScore,'choose',
                              updatedScore[i-1],
                              updatedScore[i],prevScore[i-1])

        updatedScore[i] = max(updatedScore[i-1] + indel, #Up,
                              updatedScore[i] + indel, #Left,
                              prevScore[i-1]) #Mismatch
    print(updatedScore,"\n----")
updatedScore

In [433]:
currentScore = [nullScore[1]] + updatedScore
###
residue = 'C'
for i in range(1,n+1):
    print(updatedScore)
    if residue == y[i-1]:
        print('match',currentScore[i-1] + 1)
        updatedScore[i-1] = currentScore[i-1] + 1
    else:
        print('no match')
        updatedScore[i-1] = max(updatedScore[i] + indel,
                              updatedScore[i-1] + indel)
updatedScore

[1, -4, -3, -3, -1, -1]
no match
[0, -4, -3, -3, -1, -1]
no match
[0, -4, -3, -3, -1, -1]
no match
[0, -4, -4, -3, -1, -1]
match -2
[0, -4, -4, -2, -1, -1]
no match
[0, -4, -4, -2, -2, -1]
no match


IndexError: list index out of range

In [376]:
residue = 'C'
for i in range(1,n+1):
    print(score)
    if residue == y[i-1]:
        print('match')
        score[i] += score[i-1] + 1
    else:
        print('no match')
        score[i] = max(score[i] + indel,
                       score[i-1] + indel)
score

[0 1 1 1 1 1 1]
no match
[0 1 1 1 1 1 1]
no match
[0 1 1 1 1 1 1]
no match
[0 1 1 1 1 1 1]
match
[0 1 1 1 2 1 1]
no match
[0 1 1 1 2 2 1]
no match


array([0, 1, 1, 1, 2, 2, 2])

In [377]:
residue = 'G'
for i in range(1,n+1):
    print(score)
    if residue == y[i-1]:
        print('match')
        score[i] = score[i-1] + 1
    else:
        print('no match')
        score[i] = max(score[i] + indel,
                       score[i-1] + indel)
score

[0 1 1 1 2 2 2]
no match
[0 1 1 1 2 2 2]
no match
[0 1 1 1 2 2 2]
no match
[0 1 1 1 2 2 2]
no match
[0 1 1 1 2 2 2]
no match
[0 1 1 1 2 2 2]
no match


array([0, 1, 1, 1, 2, 2, 2])

In [371]:
residue = 'G'
for i in range(1,n+1):
    print(score)
    if residue == y[i-1]:
        print('match')
        score[i] = nullScore[i-1] + 1
    else:
        print('no match')
        score[i] = max(score[i] + indel,
                       score[i-1] + indel)
score

[0 1 1 1 2 2 2]
no match
[0 1 1 1 2 2 2]
no match
[0 1 1 1 2 2 2]
no match
[0 1 1 1 2 2 2]
no match
[0 1 1 1 2 2 2]
no match
[0 1 1 1 2 2 2]
no match


array([0, 1, 1, 1, 2, 2, 2])

In [379]:
residue = 'A'
for i in range(1,n+1):
    print(score)
    if residue == y[i-1]:
        print('match')
        score[i] = score[i-1] + 1
    else:
        print('no match')
        score[i] = max(score[i] + indel,
                       score[i-1] + indel)
score

[0 1 1 1 2 1 1]
match
[0 1 1 1 2 1 1]
no match
[0 1 1 1 2 1 1]
no match
[0 1 1 1 2 1 1]
no match
[0 1 1 1 2 1 1]
match
[0 1 1 1 2 3 1]
match


array([0, 1, 1, 1, 2, 3, 4])

In [ ]:
for i in range(1,n+1):
    #print(i,score,match[i], score[i], score[i-1])
    if residue == target[i-1]:
        score[i] += score[i] + match
    else:
        score[i] = max(score[i] + indel,
                       score[i-1] + indel)
return(score)

In [334]:
def linSpace(residue,target,score,match=1,indel=0):
    n = len(y)
    for i in range(1,n+1):
        #print(i,score,match[i], score[i], score[i-1])
        if residue == target[i-1]:
            score[i] += match
        else:
            score[i] = max(score[i] + indel,
                           score[i-1] + indel)
    return(score)

In [351]:
x='ACGGAA'
y='ATTCAA'
indel = 0
score = np.cumsum([indel] + [indel for _ in y])
for _ in x:
    score = linSpace(_,y,score,match=1,indel=0)
    print(score)

score[len(y)]

[0 1 1 1 1 1 1]
[0 1 1 1 2 2 2]
[0 1 1 1 2 2 2]
[0 1 1 1 2 2 2]
[0 2 2 2 2 3 3]
[0 3 3 3 3 4 4]


4

In [341]:
x='AGTCGAG'
y='GGCTAAT'
indel = 0
score = np.cumsum([indel] + [indel for _ in y])
for _ in x:
    score = linSpace(_,y,score,match=1,indel=0)
    print(score)

score[len(y)]

[0 0 0 0 0 1 1 1]
[0 1 1 1 1 1 1 1]
[0 1 1 1 2 2 2 2]
[0 1 1 2 2 2 2 2]
[0 2 2 2 2 2 2 2]
[0 2 2 2 2 3 3 3]
[0 3 3 3 3 3 3 3]


3

In [ ]:
x='A'
y='GGCTAAT'
indel = 0
score = np.cumsum([indel] + [indel for _ in y])
for _ in x:
    score = linSpace(_,y,score,match=1,indel=0)
    print(score)

score[len(y)]

In [ ]:
x='G'
y='GGCTAAT'
indel = 0
score = np.cumsum([indel] + [indel for _ in y])
for _ in x:
    score = linSpace(_,y,score,match=1,indel=0)
    print(score)

score[len(y)]

In [342]:
x='AGTC'
y='GGCTAAT'
indel = 0
score = np.cumsum([indel] + [indel for _ in y])
for _ in x:
    score = linSpace(_,y,score,match=1,indel=0)
    print(score)

score[len(y)]

[0 0 0 0 0 1 1 1]
[0 1 1 1 1 1 1 1]
[0 1 1 1 2 2 2 2]
[0 1 1 2 2 2 2 2]


2

In [343]:
x='GAG'
y='GGCTAAT'
indel = 0
score = np.cumsum([indel] + [indel for _ in y])
for _ in x:
    score = linSpace(_,y,score,match=1,indel=0)
    print(score)

score[len(y)]

[0 1 1 1 1 1 1 1]
[0 1 1 1 1 2 2 2]
[0 2 2 2 2 2 2 2]


2